# 第3章: シングルエージェントとツール活用

## 準備

以下のセルを順番に実行して、演習に必要な環境をセットアップします。

### LLMプロバイダーの選択

このセルでは、使用するLLMプロバイダーを選択します。
`LLM_PROVIDER` 変数に、利用したいプロバイダー名を設定してください。
選択可能なプロバイダー: `"openai"`, `"azure"`, `"google"` (Vertex AI), `"google_genai"` (Gemini API), `"anthropic"`, `"bedrock"`

In [ ]:
# === LLMプロバイダーの選択 ===
# 利用したいLLMプロバイダーを以下の変数で指定してください。
# "openai", "azure", "google" (Vertex AI), "google_genai" (Gemini API), "anthropic", "bedrock" のいずれかを選択できます。
LLM_PROVIDER = "openai"  # 例: OpenAI を利用する場合

### APIキー/環境変数の設定

以下のセルを実行する前に、選択したLLMプロバイダーに応じたAPIキーまたは環境変数を設定する必要があります。

**手順:**
1.  `.env.sample` ファイルをコピーして `.env` ファイルを作成します。
2.  `.env` ファイルを開き、選択したLLMプロバイダーに対応するAPIキーや必要な情報を記述します。
    *   **OpenAI:** `OPENAI_API_KEY`
    *   **Azure OpenAI:** `AZURE_OPENAI_API_KEY`, `AZURE_OPENAI_ENDPOINT`, `OPENAI_API_VERSION`, `AZURE_OPENAI_DEPLOYMENT_NAME`
    *   **Google (Vertex AI):** `GOOGLE_CLOUD_PROJECT_ID`, `GOOGLE_CLOUD_LOCATION` (Colab環境外で実行する場合、`GOOGLE_APPLICATION_CREDENTIALS` 環境変数の設定も必要になることがあります)
    *   **Google (Gemini API):** `GOOGLE_API_KEY`
    *   **Anthropic:** `ANTHROPIC_API_KEY`
    *   **AWS Bedrock:** `AWS_ACCESS_KEY_ID`, `AWS_SECRET_ACCESS_KEY`, `AWS_REGION_NAME` (IAMロールを使用する場合は、これらのキー設定は不要な場合がありますが、リージョン名は必須です)
3.  ファイルを保存します。

**Google Colab を使用している場合:**
上記の `.env` ファイルを使用する代わりに、Colabのシークレットマネージャーに必要なキーを登録してください。
例えば、OpenAIを使用する場合は `OPENAI_API_KEY` という名前でシークレットを登録します。
Vertex AI を利用する場合は、Colab上での認証 (`google.colab.auth.authenticate_user()`) が実行されます。

このセルは、設定された情報に基づいて環境変数をロードし、LLMクライアントを初期化します。

In [ ]:
# === APIキー/環境変数の設定 ===
import os
from dotenv import load_dotenv

# .envファイルから環境変数を読み込む (存在する場合)
load_dotenv()

try:
    from google.colab import userdata
    IS_COLAB = True
except ImportError:
    IS_COLAB = False

# --- OpenAI ---
if LLM_PROVIDER == "openai":
    OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
    if not OPENAI_API_KEY and IS_COLAB:
        OPENAI_API_KEY = userdata.get("OPENAI_API_KEY")
    if not OPENAI_API_KEY:
        raise ValueError("OpenAI APIキーが設定されていません。環境変数 OPENAI_API_KEY を設定するか、Colab環境の場合はシークレットに OPENAI_API_KEY を設定してください。")
    os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

# --- Azure OpenAI ---
elif LLM_PROVIDER == "azure":
    AZURE_OPENAI_API_KEY = os.environ.get("AZURE_OPENAI_API_KEY")
    AZURE_OPENAI_ENDPOINT = os.environ.get("AZURE_OPENAI_ENDPOINT")
    OPENAI_API_VERSION = os.environ.get("OPENAI_API_VERSION")
    AZURE_OPENAI_DEPLOYMENT_NAME = os.environ.get("AZURE_OPENAI_DEPLOYMENT_NAME")

    if IS_COLAB:
        if not AZURE_OPENAI_API_KEY: AZURE_OPENAI_API_KEY = userdata.get("AZURE_OPENAI_API_KEY")
        if not AZURE_OPENAI_ENDPOINT: AZURE_OPENAI_ENDPOINT = userdata.get("AZURE_OPENAI_ENDPOINT")
        if not OPENAI_API_VERSION: OPENAI_API_VERSION = userdata.get("OPENAI_API_VERSION") # 例: "2023-07-01-preview"
        if not AZURE_OPENAI_DEPLOYMENT_NAME: AZURE_OPENAI_DEPLOYMENT_NAME = userdata.get("AZURE_OPENAI_DEPLOYMENT_NAME")

    if not AZURE_OPENAI_API_KEY: raise ValueError("Azure OpenAI APIキー (AZURE_OPENAI_API_KEY) が設定されていません。")
    if not AZURE_OPENAI_ENDPOINT: raise ValueError("Azure OpenAI エンドポイント (AZURE_OPENAI_ENDPOINT) が設定されていません。")
    if not OPENAI_API_VERSION: OPENAI_API_VERSION = "2023-07-01-preview" # デフォルトを設定することも可能
    if not AZURE_OPENAI_DEPLOYMENT_NAME: raise ValueError("Azure OpenAI デプロイメント名 (AZURE_OPENAI_DEPLOYMENT_NAME) が設定されていません。")

    os.environ["AZURE_OPENAI_API_KEY"] = AZURE_OPENAI_API_KEY
    os.environ["AZURE_OPENAI_ENDPOINT"] = AZURE_OPENAI_ENDPOINT
    os.environ["OPENAI_API_VERSION"] = OPENAI_API_VERSION

# --- Google Cloud Vertex AI (Gemini) ---
elif LLM_PROVIDER == "google":
    PROJECT_ID = os.environ.get("GOOGLE_CLOUD_PROJECT_ID") # .env 用に修正
    LOCATION = os.environ.get("GOOGLE_CLOUD_LOCATION")

    if IS_COLAB:
        if not PROJECT_ID: PROJECT_ID = userdata.get("GOOGLE_CLOUD_PROJECT_ID")
        if not LOCATION: LOCATION = userdata.get("GOOGLE_CLOUD_LOCATION") # 例: "us-central1"
        from google.colab import auth as google_auth
        google_auth.authenticate_user() # Vertex AI を使う場合は Colab での認証を推奨
    else: # Colab外の場合、.envから読み込んだ値で環境変数を設定
        if PROJECT_ID: os.environ['GOOGLE_CLOUD_PROJECT'] = PROJECT_ID # Vertex AI SDKが参照する標準的な環境変数名
        if LOCATION: os.environ['GOOGLE_CLOUD_LOCATION'] = LOCATION

    if not PROJECT_ID: raise ValueError("Google Cloud Project ID が設定されていません。環境変数 GOOGLE_CLOUD_PROJECT_ID を設定するか、Colab環境の場合はシークレットに GOOGLE_CLOUD_PROJECT_ID を設定してください。")
    if not LOCATION: LOCATION = "us-central1" # デフォルトロケーション

# --- Google Gemini API (langchain-google-genai) ---
elif LLM_PROVIDER == "google_genai":
    GOOGLE_API_KEY = os.environ.get("GOOGLE_API_KEY")
    if not GOOGLE_API_KEY and IS_COLAB:
        GOOGLE_API_KEY = userdata.get("GOOGLE_API_KEY")
    if not GOOGLE_API_KEY:
        raise ValueError("Google APIキーが設定されていません。環境変数 GOOGLE_API_KEY を設定するか、Colab環境の場合はシークレットに GOOGLE_API_KEY を設定してください。")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

# --- Anthropic (Claude) ---
elif LLM_PROVIDER == "anthropic":
    ANTHROPIC_API_KEY = os.environ.get("ANTHROPIC_API_KEY")
    if not ANTHROPIC_API_KEY and IS_COLAB:
        ANTHROPIC_API_KEY = userdata.get("ANTHROPIC_API_KEY")
    if not ANTHROPIC_API_KEY:
        raise ValueError("Anthropic APIキーが設定されていません。環境変数 ANTHROPIC_API_KEY を設定するか、Colab環境の場合はシークレットに ANTHROPIC_API_KEY を設定してください。")
    os.environ["ANTHROPIC_API_KEY"] = ANTHROPIC_API_KEY

# --- Amazon Bedrock (Claude) ---
elif LLM_PROVIDER == "bedrock":
    AWS_ACCESS_KEY_ID = os.environ.get("AWS_ACCESS_KEY_ID")
    AWS_SECRET_ACCESS_KEY = os.environ.get("AWS_SECRET_ACCESS_KEY")
    AWS_REGION_NAME = os.environ.get("AWS_REGION_NAME")

    if IS_COLAB: 
        if not AWS_ACCESS_KEY_ID: AWS_ACCESS_KEY_ID = userdata.get("AWS_ACCESS_KEY_ID")
        if not AWS_SECRET_ACCESS_KEY: AWS_SECRET_ACCESS_KEY = userdata.get("AWS_SECRET_ACCESS_KEY")
        if not AWS_REGION_NAME: AWS_REGION_NAME = userdata.get("AWS_REGION_NAME")

    if not AWS_REGION_NAME:
         raise ValueError("AWSリージョン名 (AWS_REGION_NAME) が設定されていません。Bedrock利用にはリージョン指定が必要です。")

    # 環境変数に設定 (boto3がこれらを自動で読み込む)
    if AWS_ACCESS_KEY_ID: os.environ["AWS_ACCESS_KEY_ID"] = AWS_ACCESS_KEY_ID
    if AWS_SECRET_ACCESS_KEY: os.environ["AWS_SECRET_ACCESS_KEY"] = AWS_SECRET_ACCESS_KEY
    os.environ["AWS_DEFAULT_REGION"] = AWS_REGION_NAME # boto3が参照する標準的なリージョン環境変数名
    os.environ["AWS_REGION"] = AWS_REGION_NAME # いくつかのライブラリはこちらを参照することもある

print(f"APIキー/環境変数の設定完了 (プロバイダー: {LLM_PROVIDER})")

### LLMクライアントの初期化

このセルは、上で選択・設定したLLMプロバイダーに基づいて、対応するLLMクライアントを初期化します。

In [ ]:
# === LLMクライアントの動的初期化 ===
llm = None

if LLM_PROVIDER == "openai":
    from langchain_openai import ChatOpenAI
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
elif LLM_PROVIDER == "azure":
    from langchain_openai import AzureChatOpenAI
    llm = AzureChatOpenAI(
        azure_deployment=os.environ.get("AZURE_OPENAI_DEPLOYMENT_NAME"), # 環境変数から取得
        openai_api_version=os.environ.get("OPENAI_API_VERSION"), # 環境変数から取得
        temperature=0,
    )
elif LLM_PROVIDER == "google":
    from langchain_google_vertexai import ChatVertexAI
    # PROJECT_ID, LOCATION は前のセルで環境変数に設定済みか、Colabの場合は直接利用
    llm = ChatVertexAI(model_name="gemini-2.0-flash", temperature=0, project=os.environ.get("GOOGLE_CLOUD_PROJECT"), location=os.environ.get("GOOGLE_CLOUD_LOCATION"))
elif LLM_PROVIDER == "google_genai":
    from langchain_google_genai import ChatGoogleGenerativeAI
    llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0)
elif LLM_PROVIDER == "anthropic":
    from langchain_anthropic import ChatAnthropic
    llm = ChatAnthropic(model="claude-3-haiku-20240307", temperature=0)
elif LLM_PROVIDER == "bedrock":
    from langchain_aws import ChatBedrock # langchain_community.chat_models から langchain_aws に変更の可能性あり
    # AWS_REGION_NAME は前のセルで環境変数 AWS_DEFAULT_REGION に設定済み
    llm = ChatBedrock( # BedrockChat ではなく ChatBedrock が一般的
        model_id="anthropic.claude-3-haiku-20240307-v1:0",
        # region_name=os.environ.get("AWS_DEFAULT_REGION"), # 通常、boto3が環境変数から自動で読み込む
        model_kwargs={"temperature": 0},
    )
else:
    raise ValueError(
        f"Unsupported LLM_PROVIDER: {LLM_PROVIDER}. "
        "Please choose from 'openai', 'azure', 'google', 'google_genai', 'anthropic', or 'bedrock'."
    )

print(f"LLM Provider: {LLM_PROVIDER}")
if llm:
    print(f"LLM Client Type: {type(llm)}")
    # モデル名取得の試行を汎用的に
    model_attr = (
                 getattr(llm, 'model', None) or
                 getattr(llm, 'model_name', None) or
                 getattr(llm, 'model_id', None) or
                 (hasattr(llm, 'llm') and getattr(llm.llm, 'model', None)) # 一部のLLMクライアントのネスト構造に対応
    )
    if hasattr(llm, 'azure_deployment') and not model_attr: # Azure特有の属性
        model_attr = llm.azure_deployment
        
    if model_attr:
        print(f"LLM Model: {model_attr}")
    else:
        print("LLM Model: (Could not determine model name from client attributes)")


---

### ■ 問題001: 基本的なツールの実装と ToolNode の使用

LangGraphで外部ツールを利用する最初のステップとして、シンプルなツールを定義し、それをグラフ内の `ToolNode` から呼び出す方法を学びます。ここでは、与えられた数値を二乗する簡単なPython関数をツールとして実装し、LLM（またはエージェント）からのツール呼び出し要求に応じてそのツールを実行するグラフを構築します。

*   **学習内容:**
    *   `@tool` デコレータを使ったLangChainツールの基本的な作成方法。
    *   ツール呼び出し(`ToolCall`)とツール結果(`ToolMessage`)のメッセージ形式の理解。
    *   `ToolNode` を使って、グラフ内でツールを実行する方法。
    *   LLMにツールを使わせるためのプロンプト（または `bind_tools` の利用）。

In [ ]:
# 解答欄001 - グラフ構築
from typing import ____, ____, List, Union
from langchain_core.tools import tool
from ____.____ import ____, ____
from ____.____.message import ____
from langchain_core.messages import ____, HumanMessage, AIMessage, ToolMessage, ToolCall # ToolCallを追加 (解答例より)
from uuid import uuid4 # uuid4を追加 (解答例より)

# --- 1. ツールの定義 ---
@tool
def square_number(number: float) -> float:
    """与えられた数値を二乗します。"""
    print(f"  ツール[square_number]呼び出し: number={number}")
    return number ** 2

tools = [square_number] # 作成したツールをリストに格納 (解答例より)

# --- 2. 状態の定義 ---
class BasicToolAgentState(TypedDict):
    messages: Annotated[list, add_messages]

# --- 3. ノードの定義 ---

# LLMノード (エージェントノード)
def agent_node(state: BasicToolAgentState):
    print("
[エージェントノード]")
    current_messages = state["messages"] # 解答例より
    response = None # 解答例より
    if hasattr(llm, 'bind_tools') and LLM_PROVIDER != "fake": # FakeLLMはbind_toolsを持たない場合がある (解答例より)
        llm_with_tools = llm.bind_tools(tools)
        response = llm_with_tools.invoke(current_messages)
    else:
        print("  WARN: LLMがbind_toolsをサポートしていないか、FakeLLMです。手動でツールコールを模倣します。") # 解答例より
        last_message_text = current_messages[-1].content.lower()
        if isinstance(current_messages[-1], HumanMessage) and ("二乗" in last_message_text or "square" in last_message_text): # 解答例より
            import re
            match = re.search(r'(\d+\.?\d*|\.\d+)', last_message_text) # 数値を抽出 (解答例より)
            if match:
                num_to_square = float(match.group(1))
                print(f"  FakeLLM/Fallback: キーワード検知。square_number({num_to_square}) を呼び出します。") # 解答例より
                response = AIMessage(
                    content="", 
                    tool_calls=[ToolCall(name="square_number", args={"number": num_to_square}, id=f"tool_call_{str(uuid4())[:4]}")] # 解答例より
                )
            else:
                response = AIMessage(content="FakeLLM/Fallback: 二乗する数値が見つかりませんでした。") # 解答例より
        elif isinstance(current_messages[-1], ToolMessage): # 解答例より
             response = AIMessage(content=f"ツール実行結果を受け取りました: {current_messages[-1].content}") # 解答例より
        else:
            response = llm.invoke(current_messages) # 通常のLLM呼び出し (解答例より)
            
    print(f"  エージェント応答: {response}")
    return {"messages": [response]}

# ツール実行ノード
from langgraph.prebuilt import ToolNode
tool_node = ToolNode(tools) # ToolNodeを初期化 (解答例より)

# --- 4. ルーター関数の定義 ---
def router_function(state: BasicToolAgentState) -> str:
    last_message = state["messages"][-1]
    if hasattr(last_message, "tool_calls") and last_message.tool_calls:
        print("  -> ルーター: ツール呼び出しあり。ツールノードへ。")
        return "call_tools"
    else:
        print("  -> ルーター: ツール呼び出しなし。終了。")
        return "__end__"

# --- 5. グラフの構築 ---
workflow = StateGraph(BasicToolAgentState)
workflow.add_node("agent", agent_node)
workflow.add_node("tools", tool_node) # tool_node を "tools" という名前で追加 (解答例より)

workflow.set_entry_point("agent")

workflow.add_conditional_edges(
    "agent",
    router_function,
    {
        "call_tools": "tools",
        "__end__": END
    }
)
workflow.add_edge("tools", "agent") # ツール実行後、再度エージェントノードに戻して結果を処理させる

graph_q1 = workflow.compile()

In [ ]:
# 解答欄001 - グラフ可視化
____ IPython.display ____ Image, display # 解答例より display をインポート

____:
    display(Image(graph_q1.get_graph().draw_png()))
____ Exception as e:
    print(f"グラフ描画に失敗: {e}")

In [ ]:
# 解答欄001 - グラフ実行
____:
    print("--- 基本ツールのテスト実行 (5の二乗) ---")
    initial_state_q1 = {"messages": [____(content="5を二乗して結果を教えてください。")]} # 解答例よりメッセージ変更
    thread_q1 = {"configurable": {"thread_id": "test-thread-q1"}} # 解答例より config 追加

    final_response_content = "" # 解答例より
    ____ event ____ graph_q1.____(initial_state_q1, config=thread_q1, recursion_limit=5): # 解答例より config 追加, recursion_limit は元の解答から
        print(f"Event: {event}")
        # 最後のAIMessage (ツールコールではないもの) の内容を取得 (解答例より)
        ____ 'agent' ____ event:
            agent_messages = event['agent'].get('messages', [])
            ____ agent_messages and isinstance(agent_messages[0], ____) and not agent_messages[0].____:
                final_response_content = agent_messages[0].content
        print("----")

    print(f"
最終的なAIの応答内容: {final_response_content}") # 解答例より
    if "25" in final_response_content or "25.0" in final_response_content: # 解答例より
        print("テスト成功: 応答に「25」が含まれています。") # 解答例より
    else:
        print(f"テスト失敗: 応答に「25」が含まれていません。実際の応答: {final_response_content}") # 解答例より
except Exception as e:
    print(f"エラーが発生しました: {e}")
    import traceback
    traceback.print_exc()

<details><summary>解答001</summary>

``````python
from typing import TypedDict, Annotated, List, Union
from langchain_core.tools import tool
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, ToolMessage, ToolCall
from IPython.display import Image, display

# --- 1. ツールの定義 ---
@tool
def square_number(number: float) -> float:
    """与えられた数値を二乗します。"""
    print(f"  ツール[square_number]呼び出し: number={number}")
    return number ** 2

tools = [square_number]

# --- 2. 状態の定義 ---
class BasicToolAgentState(TypedDict):
    messages: Annotated[list, add_messages]

# --- 3. ノードの定義 ---
def agent_node(state: BasicToolAgentState):
    print("
[エージェントノード]")
    current_messages = state["messages"]
    response = None
    if hasattr(llm, 'bind_tools') and LLM_PROVIDER != "fake": # FakeLLMはbind_toolsを持たない場合がある
        llm_with_tools = llm.bind_tools(tools)
        response = llm_with_tools.invoke(current_messages)
    else:
        print("  WARN: LLMがbind_toolsをサポートしていないか、FakeLLMです。手動でツールコールを模倣します。")
        last_message_text = current_messages[-1].content.lower()
        if isinstance(current_messages[-1], HumanMessage) and ("二乗" in last_message_text or "square" in last_message_text):
            import re
            match = re.search(r'(\d+\.?\d*|\.\d+)', last_message_text) # 数値を抽出
            if match:
                num_to_square = float(match.group(1))
                print(f"  FakeLLM/Fallback: キーワード検知。square_number({num_to_square}) を呼び出します。")
                response = AIMessage(
                    content="", 
                    tool_calls=[ToolCall(name="square_number", args={"number": num_to_square}, id=f"tool_call_{str(uuid4())[:4]}")]
                )
            else:
                response = AIMessage(content="FakeLLM/Fallback: 二乗する数値が見つかりませんでした。")
        elif isinstance(current_messages[-1], ToolMessage):
             response = AIMessage(content=f"ツール実行結果を受け取りました: {current_messages[-1].content}")
        else:
            response = llm.invoke(current_messages) # 通常のLLM呼び出し
            
    print(f"  エージェント応答: {response}")
    return {"messages": [response]}

from langgraph.prebuilt import ToolNode
tool_node = ToolNode(tools)

# --- 4. ルーター関数の定義 ---
def router_function(state: BasicToolAgentState) -> str:
    last_message = state["messages"][-1]
    if hasattr(last_message, "tool_calls") and last_message.tool_calls:
        print("  -> ルーター: ツール呼び出しあり。ツールノードへ。")
        return "call_tools"
    else:
        print("  -> ルーター: ツール呼び出しなし。終了。")
        return "__end__"

# --- 5. グラフの構築 ---
workflow = StateGraph(BasicToolAgentState)
workflow.add_node("agent", agent_node)
workflow.add_node("tools", tool_node)

workflow.set_entry_point("agent")

workflow.add_conditional_edges(
    "agent",
    router_function,
    {
        "call_tools": "tools",
        "__end__": END
    }
)
workflow.add_edge("tools", "agent")

graph_q1 = workflow.compile()
try:
    display(Image(graph_q1.get_graph().draw_png()))
except Exception as e:
    print(f"グラフ描画に失敗: {e}")

# --- 6. グラフの実行 --- 
print("--- 基本ツールのテスト実行 (5の二乗) ---")
initial_state_q1 = {"messages": [HumanMessage(content="5を二乗して結果を教えてください。")]}
thread_q1 = {"configurable": {"thread_id": "test-thread-q1"}}}

final_response_content = ""
for event in graph_q1.stream(initial_state_q1, config=thread_q1, recursion_limit=5):
    print(f"Event: {event}")
    # 最後のAIMessage (ツールコールではないもの) の内容を取得
    if 'agent' in event:
        agent_messages = event['agent'].get('messages', [])
        if agent_messages and isinstance(agent_messages[0], AIMessage) and not agent_messages[0].tool_calls:
            final_response_content = agent_messages[0].content
    print("----")

print(f"
最終的なAIの応答内容: {final_response_content}")
if "25" in final_response_content or "25.0" in final_response_content:
    print("テスト成功: 応答に「25」が含まれています。")
else:
    print(f"テスト失敗: 応答に「25」が含まれていません。実際の応答: {final_response_content}")
``````
</details>

<details><summary>解説001</summary>

#### この問題のポイント

*   **`@tool`デコレータ:** Python関数に`@tool`デコレータを付けるだけで、LangChainが認識できるツールとして簡単に定義できます。関数のdocstringはツールの説明としてLLMに提供され、型ヒントはLLMがツールを正しく呼び出すための引数の型情報として利用されます。
*   **`ToolNode`:** `langgraph.prebuilt.ToolNode` は、渡されたツールのリスト（この場合は `[square_number]`）を実行するための専用ノードです。LLM（エージェントノード）がツール呼び出しを含む`AIMessage`を返すと、その`tool_calls`属性に基づいて`ToolNode`が対応するツールを実行し、結果を`ToolMessage`として返します。
*   **LLMへのツールのバインド:** `llm.bind_tools(tools)` を使うことで、LLM（特にOpenAIのFunction Calling対応モデルなど）に対して利用可能なツールを明示的に伝えることができます。これにより、LLMはユーザーの指示に応じて適切なツールを選択し、必要な引数と共に呼び出す形式の `AIMessage` (具体的には `tool_calls` 属性を持つ) を生成するようになります。
    *   `bind_tools` をサポートしていないLLMや、FakeLLMを使用している場合は、この機能が期待通りに動作しないことがあります。その場合のフォールバックとして、この解答例では入力テキストのキーワードに基づいて手動でツールコールを模倣するロジックを入れていますが、これはあくまでデモ用です。
*   **グラフのフロー:**
    1.  `agent_node`がユーザー入力に基づいて応答を生成します。ツールが必要だと判断すれば、`tool_calls`を含む`AIMessage`を返します。
    2.  `router_function`が`AIMessage`に`tool_calls`が含まれているかを確認し、含まれていれば処理を`tools` (ToolNode) に分岐します。
    3.  `ToolNode`がツール（`square_number`）を実行し、結果を`ToolMessage`として返します。
    4.  `ToolNode`の処理後、エッジは再び`agent_node`に戻ります。`agent_node`は`ToolMessage`（ツールの実行結果）を含む更新された状態を受け取り、それに基づいて最終的な応答をユーザーに返します。
    5.  最終的な応答にツール呼び出しが含まれていなければ、ルーターは処理を`END`に分岐させ、グラフが終了します。
*   **状態 (`messages`):** 会話の履歴（`HumanMessage`, `AIMessage`, `ToolMessage`）が`messages`キーに蓄積されていき、各ノードはこの履歴を参照して次のアクションを決定します。

---</details>

### ■ 問題002: ReAct 風エージェント - LLM によるツール使用判断と応答選択

より自律的なエージェントの振る舞いを実現するために、LLM自身に「次にどのツールを使うべきか」または「ユーザーに直接応答すべきか」を判断させるReAct (Reasoning and Acting) 風のロジックを組み込みます。ここでは、LLMが状況に応じてツール（例: 前問の`square_number`や、新たに定義する簡単な文字列操作ツール）の使用を決定し、その結果を元に応答を生成する、というサイクルを実装します。

*   **学習内容:**
    *   LLMのプロンプトを工夫し、ツールを使うべきか、どのツールを使うべきか、あるいは直接応答すべきかを考えさせる方法。
    *   ツール呼び出し(`tool_calls`)の有無によって、グラフの次の遷移先（ツール実行ノード or 終了ノード）を動的に決定するルーターの実装。
    *   ツール実行後、その結果 (`ToolMessage`) をLLMに渡し、最終的な応答を生成させる流れ。

In [ ]:
# 解答欄002 - グラフ構築
from langchain_core.tools import tool
from ____.____ import ____, ____ # ENDを追加 (解答例より)
from ____.____.message import ____
from langchain_core.messages import ____, ____, ____, ToolMessage, ToolCall # BaseMessage, ToolCallを追加 (解答例より)
from typing import TypedDict, Annotated, List # TypedDict, Annotated, List を追加 (解答例より)
from uuid import uuid4
from langgraph.prebuilt import ToolNode # ToolNodeをインポート (解答例より)

# --- 1. ツールの追加定義 ---
@tool
def reverse_string(text: str) -> str:
    """与えられた文字列を逆順にします。"""
    print(f"  ツール[reverse_string]呼び出し: text='{text}'")
    return text[::-1]

tools_q2 = [square_number, reverse_string] # square_numberは問題001から再利用 (解答例より)

# --- 2. 状態の定義 (問題001と同じでOK) ---
# class BasicToolAgentState(TypedDict): messages: Annotated[list, add_messages] (解答例よりクラス定義はコメントアウト)

# --- 3. ノードの定義 ---
def react_agent_node(state: BasicToolAgentState):
    print("
[ReActエージェントノード]")
    current_messages = state["messages"]
    response = None

    # OpenAI Function Calling対応モデルや、ツール利用をサポートするLLMを想定
    if hasattr(llm, 'bind_tools') and LLM_PROVIDER != "fake":
        llm_with_tools_q2 = llm.bind_tools(tools_q2)
        response = llm_with_tools_q2.invoke(current_messages)
    else:
        print("  WARN: LLMがbind_toolsをサポートしていないかFakeLLMです。手動でツールコールを模倣します。")
        last_message = current_messages[-1] # 解答例より
        tool_called_in_fallback = False # 解答例より
        if isinstance(last_message, HumanMessage): # 解答例より
            text_content = last_message.content.lower() # 解答例より
            import re # 解答例より
            if "二乗" in text_content or "square" in text_content: # 解答例より
                match = re.search(r'(\d+\.?\d*|\.\d+)', text_content) # 解答例より
                if match: # 解答例より
                    num = float(match.group(1)) # 解答例より
                    response = AIMessage(content="", tool_calls=[ToolCall(name="square_number", args={"number": num}, id=f"ftc_sq_{uuid4()[:4]}")]) # 解答例より
                    tool_called_in_fallback = True # 解答例より
            elif "逆順" in text_content or "reverse" in text_content: # 解答例より
                match = re.search(r"['"]([^'"]*)['"]", text_content) # クォートされた文字列を優先 (解答例より)
                str_to_rev = match.group(1) if match else text_content.replace("逆順","").replace("reverse","").strip() # 解答例より
                response = AIMessage(content="", tool_calls=[ToolCall(name="reverse_string", args={"text": str_to_rev}, id=f"ftc_rev_{uuid4()[:4]}")]) # 解答例より
                tool_called_in_fallback = True # 解答例より
        
        if not tool_called_in_fallback: # 解答例より
            if isinstance(last_message, ToolMessage): # 解答例より
                response = AIMessage(content=f"FakeLLM: ツール「{last_message.name}」の結果「{last_message.content}」を受け取りました。これを元に応答します。") # 解答例より
            else: # 解答例より
                response = AIMessage(content="FakeLLM: こんにちは！ツールは不要と判断しました。") # 解答例より

    print(f"  エージェント応答: {response}")
    return {"messages": [response]}

tool_node_q2 = ToolNode(tools_q2) # tools_q2 を使用 (解答例より)

# ルーター関数 (問題001と同じでOK)
# def router_function(state: BasicToolAgentState) -> str: ... (解答例よりコメントアウト)

# --- 4. グラフの構築 ---
workflow_q2 = StateGraph(BasicToolAgentState)
workflow_q2.add_node("agent", react_agent_node)
workflow_q2.add_node("tools", tool_node_q2)

workflow_q2.set_entry_point("agent")
workflow_q2.add_conditional_edges("agent", router_function, {"call_tools": "tools", "__end__": END})
workflow_q2.add_edge("tools", "agent")

graph_q2 = workflow_q2.compile()

In [ ]:
# 解答欄002 - グラフ可視化
____ IPython.display ____ Image, display # 解答例より display をインポート

____:
    display(Image(graph_q2.get_graph().draw_png()))
____ Exception as e:
    print(f"グラフ描画に失敗: {e}")

In [ ]:
# 解答欄002 - グラフ実行
thread_q2_1 = {"configurable": {"thread_id": "test-thread-q2-sq"}} # 解答例より
print("
--- ReAct風エージェントテスト (square_number: 12の二乗) ---") # 解答例よりメッセージ変更
initial_state_q2_sq = {"messages": [____(content="数値12を二乗して、その結果を教えてください。")]} # 解答例よりメッセージ変更
____ event ____ graph_q2.____(initial_state_q2_sq, config=thread_q2_1, recursion_limit=5): print(f"Event: {event}
----"); # 解答例より config 追加, recursion_limit は元の解答から

thread_q2_2 = {"configurable": {"thread_id": "test-thread-q2-rev"}} # 解答例より
print("
--- ReAct風エージェントテスト (reverse_string: 'LangGraph'を逆順に) ---") # 解答例よりメッセージ変更
initial_state_q2_rev = {"messages": [____(content="'LangGraph'という文字列を逆順にして、その結果を教えてください。")]} # 解答例よりメッセージ変更
____ event ____ graph_q2.____(initial_state_q2_rev, config=thread_q2_2, recursion_limit=5): print(f"Event: {event}
----"); # 解答例より config 追加, recursion_limit は元の解答から

thread_q2_3 = {"configurable": {"thread_id": "test-thread-q2-no-tool"}} # 解答例より
print("
--- ReAct風エージェントテスト (ツール不要: こんにちは) ---") # 解答例よりメッセージ変更
initial_state_q2_no_tool = {"messages": [____(content="こんにちは、調子はどうですか？")]}
for event in graph_q2.____(initial_state_q2_no_tool, config=thread_q2_3, recursion_limit=5): print(f"Event: {event}
----"); # 解答例より config 追加, recursion_limit は元の解答から

<details><summary>解答002</summary>

``````python
from langchain_core.tools import tool
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, ToolMessage, ToolCall
from typing import TypedDict, Annotated, List
from uuid import uuid4
from IPython.display import Image, display
from langgraph.prebuilt import ToolNode # ToolNodeをインポート

# --- 1. ツールの追加定義 ---
# square_number は問題001で定義済みなので再利用

@tool
def reverse_string(text: str) -> str:
    """与えられた文字列を逆順にします。"""
    print(f"  ツール[reverse_string]呼び出し: text='{text}'")
    return text[::-1]

tools_q2 = [square_number, reverse_string] 

# --- 2. 状態の定義 (問題001と同じ) ---
class BasicToolAgentState(TypedDict):
    messages: Annotated[list, add_messages]

# --- 3. ノードの定義 ---
def react_agent_node(state: BasicToolAgentState):
    print("
[ReActエージェントノード]")
    current_messages = state["messages"]
    response = None
    
    if hasattr(llm, 'bind_tools') and LLM_PROVIDER != "fake":
        llm_with_tools_q2 = llm.bind_tools(tools_q2)
        # 実際のLLMなら、ここでシステムプロンプトにReAct的な指示を入れるとより効果的
        # 例: "You are a helpful assistant that can use tools. First think step-by-step if a tool is needed. Then call the tool or respond directly."
        response = llm_with_tools_q2.invoke(current_messages)
    else: # FakeLLMやbind_tools非対応LLM用のフォールバック
        print("  WARN: LLMがbind_toolsをサポートしていないかFakeLLMです。手動でツールコールを模倣します。")
        last_message = current_messages[-1]
        tool_called_in_fallback = False
        if isinstance(last_message, HumanMessage):
            text_content = last_message.content.lower()
            import re
            if "二乗" in text_content or "square" in text_content:
                match = re.search(r'(\d+\.?\d*|\.\d+)', text_content)
                if match:
                    num = float(match.group(1))
                    response = AIMessage(content="", tool_calls=[ToolCall(name="square_number", args={"number": num}, id=f"ftc_sq_{uuid4()[:4]}")])
                    tool_called_in_fallback = True
            elif "逆順" in text_content or "reverse" in text_content:
                match = re.search(r"['"]([^'"]*)['"]", text_content) # クォートされた文字列を優先
                str_to_rev = match.group(1) if match else text_content.replace("逆順","").replace("reverse","").strip()
                response = AIMessage(content="", tool_calls=[ToolCall(name="reverse_string", args={"text": str_to_rev}, id=f"ftc_rev_{uuid4()[:4]}")])
                tool_called_in_fallback = True
        
        if not tool_called_in_fallback:
            if isinstance(last_message, ToolMessage):
                response = AIMessage(content=f"FakeLLM: ツール「{last_message.name}」の結果「{last_message.content}」を受け取りました。これを元に応答します。")
            else:
                response = AIMessage(content="FakeLLM: こんにちは！ツールは不要と判断しました。")
    
    print(f"  エージェント応答: {response}")
    return {"messages": [response]}

tool_node_q2 = ToolNode(tools_q2) 

# ルーター関数 (問題001と同じものを使用)
def router_function(state: BasicToolAgentState) -> str:
    last_message = state["messages"][-1]
    if hasattr(last_message, "tool_calls") and last_message.tool_calls:
        print("  -> ルーター: ツール呼び出しあり。ツールノードへ。")
        return "call_tools"
    else:
        print("  -> ルーター: ツール呼び出しなし。終了。")
        return "__end__"

# --- 4. グラフの構築 ---
workflow_q2 = StateGraph(BasicToolAgentState)
workflow_q2.add_node("agent", react_agent_node)
workflow_q2.add_node("tools", tool_node_q2)

workflow_q2.set_entry_point("agent")
workflow_q2.add_conditional_edges("agent", router_function, {"call_tools": "tools", "__end__": END})
workflow_q2.add_edge("tools", "agent")

graph_q2 = workflow_q2.compile()
try:
    display(Image(graph_q2.get_graph().draw_png()))
except Exception as e:
    print(f"グラフ描画に失敗: {e}")

# --- 5. グラフの実行 --- 
thread_q2_1 = {"configurable": {"thread_id": "test-thread-q2-sq"}}}
print("
--- ReAct風エージェントテスト (square_number: 12の二乗) ---")
initial_state_q2_sq = {"messages": [HumanMessage(content="数値12を二乗して、その結果を教えてください。")]}
for event in graph_q2.stream(initial_state_q2_sq, config=thread_q2_1, recursion_limit=5): print(f"Event: {event}
----");

thread_q2_2 = {"configurable": {"thread_id": "test-thread-q2-rev"}}}
print("
--- ReAct風エージェントテスト (reverse_string: 'LangGraph'を逆順に) ---")
initial_state_q2_rev = {"messages": [HumanMessage(content="'LangGraph'という文字列を逆順にして、その結果を教えてください。")]}
for event in graph_q2.stream(initial_state_q2_rev, config=thread_q2_2, recursion_limit=5): print(f"Event: {event}
----");

thread_q2_3 = {"configurable": {"thread_id": "test-thread-q2-no-tool"}}}
print("
--- ReAct風エージェントテスト (ツール不要: こんにちは) ---")
initial_state_q2_no_tool = {"messages": [HumanMessage(content="こんにちは、調子はどうですか？")]}
for event in graph_q2.stream(initial_state_q2_no_tool, config=thread_q2_3, recursion_limit=5): print(f"Event: {event}
----");
``````
</details>

<details><summary>解説002</summary>

#### この問題のポイント

*   **LLMによる判断の重要性:** ReAct（Reasoning and Acting）パラダイムの核心は、LLM自身が次のアクション（ツールを使うか、どのツールを使うか、ユーザーに応答するかなど）を思考し、決定することです。これを実現するためには、LLMにその能力を発揮させるようなプロンプト設計が重要になります。例えば、システムプロンプトに「あなたはタスクを達成するために利用可能なツールを計画的に使うことができます。まず現在の状況と目標を考慮し、次に取るべきアクションを決定してください。アクションは、ツールの利用か、ユーザーへの最終応答です。」といった指示を含めることが考えられます。
*   **ツールリストの提供:** `llm.bind_tools(tools_q2)` のように、利用可能なツールのリストをLLMに提供することで、LLMはそれらのツールの中から適切なものを選択しようとします。各ツールの説明（docstring）が、LLMがツールを正しく理解し選択する上で非常に重要です。
*   **ルーティングロジック:** `router_function` は、LLM（エージェントノード）の応答に `tool_calls` が含まれているかどうかを判断します。
    *   `tool_calls` があれば、次にツール実行ノード (`tool_node_q2`) に処理を移します。
    *   `tool_calls` がなければ、LLMは直接ユーザーに応答しようとしていると判断し、処理を終了 (`END`) させます。
*   **処理サイクル:**
    1.  ユーザー入力がエージェントノードに渡されます。
    2.  エージェントノード内のLLMが、入力と会話履歴、利用可能なツール情報に基づいて、ツールを使うか最終応答を返すかを決定します。
    3.  ツールを使うと判断した場合、`tool_calls` を含む `AIMessage` を返します。
    4.  ルーターがこれを検知し、ツールノードに処理を渡します。
    5.  ツールノードが指定されたツールを実行し、結果を `ToolMessage` として返します。
    6.  この `ToolMessage` を含む更新された会話履歴が、再びエージェントノードに渡されます。
    7.  LLMはツールの実行結果を踏まえて、最終的な応答を生成するか、さらに別のツールを使うかなどを判断します。
    8.  最終的にLLMがツールを使わずに応答を返すと判断した場合、ルーターは処理を終了させます。
*   **FakeLLMの限界:** 解答例のフォールバック処理で示されているように、`bind_tools` を完全には模倣できないFakeLLMや、ツール利用を前提としない単純なLLMでは、ReAct的な挙動をさせるのは難しいです。実際のReActエージェントでは、OpenAIのFunction Calling対応モデルや、LangChainのAgent Executorと連携可能なモデルなど、ツール利用を高度にサポートするLLMの利用が一般的です。

---</details>

### ■ 問題003: 複数ツールからの選択

エージェントが複数のツールを利用可能な状況で、現在のタスクや会話の文脈に応じて、LLMがそれらの中から最も適切なツールを一つ（または複数）選択して使用する能力は非常に重要です。この問題では、複数の異なるツール（例: `square_number`、`reverse_string`、そして新たにウェブ検索を行う`search_tool`）をエージェントに提供し、LLMがユーザーの質問に応じて適切なツールを選択・実行するグラフを構築します。

*   **学習内容:**
    *   複数のツールをリストとしてLLMに提供する方法。
    *   LLMがユーザーの意図を解釈し、提供されたツールの中から最適なものを選択する（またはツールを使わないと判断する）思考プロセス（のシミュレーションや期待）。
    *   選択されたツールに応じた引数をLLMが正しく生成する能力（の期待）。
    *   Tavily Search API ( `langchain_community.tools.tavily_search.TavilySearchResults` ) のような外部連携ツールをグラフに組み込む基本的な方法。

In [ ]:
# 解答欄003 - グラフ構築
from langchain_core.tools import tool # 解答例より tool をインポート (既にインポート済みだが明示)
from ____.____ import ____, ____ # 解答例より ____ をインポート (既にインポート済みだが明示)
from ____.____.message import ____ # 解答例より (既にインポート済みだが明示)
from langchain_core.messages import ____, ____, AIMessage, ToolMessage, ToolCall # 解答例より (既にインポート済みだが明示)
from typing import TypedDict, Annotated, List # 解答例より (既にインポート済みだが明示)
from uuid import uuid4 # 解答例より (既にインポート済みだが明示)
from langgraph.prebuilt import ToolNode # 解答例より (既にインポート済みだが明示)

# search_tool は準備セルで TavilySearchResults またはダミーとして初期化済み
# square_number, reverse_string も前の問題で定義済み

tools_q3 = [square_number, reverse_string, search_tool] # search_tool を追加 (解答例より)

# 状態定義 (BasicToolAgentStateを再利用)
# class BasicToolAgentState(TypedDict): messages: Annotated[list, add_messages] (解答例よりコメントアウト)

# ノード定義
def multi_tool_agent_node(state: BasicToolAgentState):
    print("
[マルチツールエージェントノード]")
    current_messages = state["messages"]
    response = None

    if hasattr(llm, 'bind_tools') and LLM_PROVIDER != "fake":
        llm_with_tools_q3 = llm.bind_tools(tools_q3)
        response = llm_with_tools_q3.invoke(current_messages)
    else:
        print("  WARN: LLMがbind_toolsをサポートしていないかFakeLLMです。手動でツールコールを模倣します。")
        last_message = current_messages[-1]
        tool_called_in_fallback = False
        if isinstance(last_message, HumanMessage):
            text_content = last_message.content.lower()
            import re
            # 検索ツールの呼び出し模倣 (Tavilyが利用可能か、またはダミーかで挙動が変わる) (解答例より)
            if ("天気" in text_content or "検索" in text_content or "調べて" in text_content) and search_tool is not None: # 解答例より
                query = text_content
                if "天気" in text_content: query = text_content.replace("教えて","").strip("?？ ") # 解答例より
                response = AIMessage(content="", tool_calls=[ToolCall(name=search_tool.name, args={"query": query}, id=f"ftc_search_{uuid4()[:4]}")]) # 解答例より
                tool_called_in_fallback = True
            elif "二乗" in text_content:
                match = re.search(r'(\d+\.?\d*|\.\d+)', text_content)
                if match: 
                    response = AIMessage(content="", tool_calls=[ToolCall(name="square_number", args={"number": float(match.group(1))}, id=f"ftc_sq_{uuid4()[:4]}")])
                    tool_called_in_fallback = True
            elif "逆順" in text_content: # 解答例より
                match = re.search(r"['"]([^'"]*)['"]", text_content) # 解答例より
                str_to_rev = match.group(1) if match else text_content.replace("逆順","").strip() # 解答例より
                response = AIMessage(content="", tool_calls=[ToolCall(name="reverse_string", args={"text": str_to_rev}, id=f"ftc_rev_{uuid4()[:4]}")]) # 解答例より
                tool_called_in_fallback = True # 解答例より
        
        if not tool_called_in_fallback:
            if isinstance(last_message, ToolMessage):
                response = AIMessage(content=f"FakeLLM: ツール「{last_message.name}」の結果「{last_message.content[:100]}...」を元に最終応答を生成します。") # 解答例より
            else:
                response = AIMessage(content="FakeLLM: どのツールも適切でないと判断しました。通常の応答を返します。") # 解答例より
    
    print(f"  エージェント応答: {response}")
    return {"messages": [response]}

tool_node_q3 = ToolNode(tools_q3) # tools_q3 を使用 (解答例より)

# ルーター関数 (router_functionを再利用)
# def router_function(state: BasicToolAgentState) -> str: ... (解答例よりコメントアウト)

# グラフ構築
workflow_q3 = StateGraph(BasicToolAgentState)
workflow_q3.add_node("agent", multi_tool_agent_node)
workflow_q3.add_node("tools", tool_node_q3)

workflow_q3.set_entry_point("agent")
workflow_q3.add_conditional_edges("agent", router_function, {"call_tools": "tools", "__end__": END})
workflow_q3.add_edge("tools", "agent")

graph_q3 = workflow_q3.compile()

In [ ]:
# 解答欄003 - グラフ可視化
____ IPython.display ____ Image, display # 解答例より display をインポート

____:
    display(Image(graph_q3.get_graph().draw_png()))
____ Exception as e:
    print(f"グラフ描画に失敗: {e}")

In [ ]:
# 解答欄003 - グラフ実行
queries_q3 = [
    "日本の首都、東京の今日の天気を教えてください。",
    "数値 7 を二乗するといくつになりますか？",
    "'supercalifragilisticexpialidocious' を逆順にするとどうなりますか？",
    "今日の気分はどうですか？" # ツール不要な質問
]

____ i, q ____ enumerate(queries_q3):
    print(f"
--- マルチツール選択テスト {i+1} (入力: {q}) ---")
    ____ "天気" ____ q.lower() ____ (search_tool ____ None or (hasattr(search_tool, 'name') and search_tool.name == "dummy_search_tool")) and TAVILY_API_KEY is None: # 解答例より
        print("  TAVILY_API_KEY が未設定のため、天気検索はダミー応答になります。") # 解答例より

    thread_q3 = {"configurable": {"thread_id": f"test-thread-q3-{i}-{uuid4()[:4]}"}} # よりユニークなID (解答例より)
    initial_state_q3 = {"messages": [____(content=q)]}
    final_content_q3 = "(応答なし)" # 解答例より
    for event in graph_q3.____(initial_state_q3, config=thread_q3, recursion_limit=5): # 解答例より config 追加, recursion_limit は元の解答から
        print(f"Event: {event}")
        if 'agent' in event: # 解答例より
            agent_messages = event['agent'].get('messages', [])
            if agent_messages and isinstance(agent_messages[0], ____) and not agent_messages[0].____:
                final_content_q3 = agent_messages[0].content
        print("----");
    print(f"最終応答内容: {final_content_q3}")

<details><summary>解答003</summary>

``````python
from langchain_core.tools import tool
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, ToolMessage, ToolCall
from typing import TypedDict, Annotated, List
from uuid import uuid4
from IPython.display import Image, display
from langgraph.prebuilt import ToolNode

# search_tool は準備セルで TavilySearchResults またはダミーとして初期化済み
# square_number, reverse_string も前の問題で定義済み

tools_q3 = [square_number, reverse_string, search_tool] 

# 状態定義 (BasicToolAgentStateを再利用)
# class BasicToolAgentState(TypedDict): messages: Annotated[list, add_messages]

# ノード定義
def multi_tool_agent_node(state: BasicToolAgentState):
    print("
[マルチツールエージェントノード]")
    current_messages = state["messages"]
    response = None

    if hasattr(llm, 'bind_tools') and LLM_PROVIDER != "fake":
        llm_with_tools_q3 = llm.bind_tools(tools_q3)
        response = llm_with_tools_q3.invoke(current_messages)
    else:
        print("  WARN: LLMがbind_toolsをサポートしていないかFakeLLMです。手動でツールコールを模倣します。")
        last_message = current_messages[-1]
        tool_called_in_fallback = False
        if isinstance(last_message, HumanMessage):
            text_content = last_message.content.lower()
            import re
            # 検索ツールの呼び出し模倣 (Tavilyが利用可能か、またはダミーかで挙動が変わる)
            if ("天気" in text_content or "検索" in text_content or "調べて" in text_content) and search_tool is not None:
                query = text_content
                if "天気" in text_content: query = text_content.replace("教えて","").strip("?？ ")
                # search_tool.name は TavilySearchResults インスタンスなら 'tavily_search_results_json'
                # ダミーなら 'dummy_search_tool'
                response = AIMessage(content="", tool_calls=[ToolCall(name=search_tool.name, args={"query": query}, id=f"ftc_search_{uuid4()[:4]}")])
                tool_called_in_fallback = True
            elif "二乗" in text_content:
                match = re.search(r'(\d+\.?\d*|\.\d+)', text_content)
                if match: 
                    response = AIMessage(content="", tool_calls=[ToolCall(name="square_number", args={"number": float(match.group(1))}, id=f"ftc_sq_{uuid4()[:4]}")])
                    tool_called_in_fallback = True
            elif "逆順" in text_content:
                match = re.search(r"['"]([^'"]*)['"]", text_content)
                str_to_rev = match.group(1) if match else text_content.replace("逆順","").strip()
                response = AIMessage(content="", tool_calls=[ToolCall(name="reverse_string", args={"text": str_to_rev}, id=f"ftc_rev_{uuid4()[:4]}")])
                tool_called_in_fallback = True
        
        if not tool_called_in_fallback:
            if isinstance(last_message, ToolMessage):
                response = AIMessage(content=f"FakeLLM: ツール「{last_message.name}」の結果「{last_message.content[:100]}...」を元に最終応答を生成します。")
            else:
                response = AIMessage(content="FakeLLM: どのツールも適切でないと判断しました。通常の応答を返します。")
    
    print(f"  エージェント応答: {response}")
    return {"messages": [response]}

tool_node_q3 = ToolNode(tools_q3)

# ルーター関数 (router_functionを再利用)
# def router_function(state: BasicToolAgentState) -> str: ... (問題001, 002と同じ)

# グラフ構築
workflow_q3 = StateGraph(BasicToolAgentState)
workflow_q3.add_node("agent", multi_tool_agent_node)
workflow_q3.add_node("tools", tool_node_q3)

workflow_q3.set_entry_point("agent")
workflow_q3.add_conditional_edges("agent", router_function, {"call_tools": "tools", "__end__": END})
workflow_q3.add_edge("tools", "agent")

graph_q3 = workflow_q3.compile()
try:
    display(Image(graph_q3.get_graph().draw_png()))
except Exception as e:
    print(f"グラフ描画に失敗: {e}")

# 実行テスト
queries_q3 = [
    "日本の首都、東京の今日の天気を教えてください。",
    "数値 7 を二乗するといくつになりますか？",
    "'supercalifragilisticexpialidocious' を逆順にするとどうなりますか？",
    "今日の気分はどうですか？" 
]

for i, q in enumerate(queries_q3):
    print(f"
--- マルチツール選択テスト {i+1} (入力: {q}) ---")
    if "天気" in q.lower() and (search_tool is None or (hasattr(search_tool, 'name') and search_tool.name == "dummy_search_tool")) and TAVILY_API_KEY is None:
        print("  TAVILY_API_KEY が未設定のため、天気検索はダミー応答になります。")

    thread_q3 = {"configurable": {"thread_id": f"test-thread-q3-{i}-{uuid4()[:4]}"}} # よりユニークなID
    initial_state_q3 = {"messages": [HumanMessage(content=q)]}
    final_content_q3 = "(応答なし)"
    for event in graph_q3.stream(initial_state_q3, config=thread_q3, recursion_limit=5):
        print(f"Event: {event}")
        if 'agent' in event:
            agent_messages = event['agent'].get('messages', [])
            if agent_messages and isinstance(agent_messages[0], AIMessage) and not agent_messages[0].tool_calls:
                final_content_q3 = agent_messages[0].content
        print("----");
    print(f"最終応答内容: {final_content_q3}")
``````
</details>

<details><summary>解説003</summary>

#### この問題のポイント

*   **ツールセットの拡張:** `tools_q3 = [square_number, reverse_string, search_tool]` のように、エージェントが利用できるツールをリストとして定義し、これをLLMと`ToolNode`に提供します。`search_tool` は準備セルで `TavilySearchResults` またはそのダミーとして初期化されています。
*   **LLMによるツール選択:** 理想的には、`multi_tool_agent_node` 内のLLM (`llm.bind_tools(tools_q3).invoke(...)`) が、ユーザーの質問の意図を理解し、`tools_q3` の中から最も適切なツール（またはツール不要と判断）を選択し、そのツールに必要な引数を生成して `tool_calls` に含めます。
    *   例えば、「東京の天気は？」という質問なら `search_tool` を、「10を二乗して」なら `square_number` を選択することが期待されます。
    *   ツールのdocstringと型ヒントが、ここでもLLMの正しいツール選択と引数生成に役立ちます。
*   **Tavily Search APIの利用:** `TavilySearchResults` は、ウェブ検索を行うためのツールです。APIキー (`TAVILY_API_KEY`) が正しく設定されていれば、実際にウェブ検索を行いその結果を返します。APIキーがない場合やダミーのツールが使われている場合は、その旨を示すメッセージが返るか、あるいはエラーになります（この解答例ではダミーが機能するようにしています）。
*   **グラフ構造と処理フロー:** グラフの構造自体は問題001や002とほぼ同じですが、エージェントがより多くのツールを扱えるようになった点が異なります。LLMがどのツールを選択するかによって、`ToolNode`で実行される具体的な処理が変わります。
*   **FakeLLMでの模倣の複雑化:** `LLM_PROVIDER = "fake"` の場合のフォールバックロジックは、複数のツールに対応するためにさらに複雑になります。どのツールを呼び出すべきか、そしてその引数をどう抽出するかをルールベースで模倣する必要があり、これは実際のLLMの能力を完全に再現するものではありません。あくまで、グラフの基本的な流れとツール連携の概念を理解するためものと捉えてください。

---</details>

### ■ 問題004: ツール呼び出しの強制 (Forced Tool Calling)

特定の状況やタスクにおいて、エージェントに特定のツールを必ず使用させたい場合があります。LangChainの一部のLLM連携機能では、ツール呼び出しを「強制」するオプションが提供されています。この問題では、LLMに対して特定のツール（例: `square_number`）の使用を強制し、ユーザー入力の内容に関わらずそのツールが呼び出されるようにグラフを構築します。

*   **学習内容:**
    *   LLMの `bind_tools` メソッド（または類似の機能）において、特定のツール呼び出しを強制する `tool_choice` パラメータ（または等価のオプション）の利用方法（主にOpenAIのモデルで顕著）。
    *   強制ツール呼び出しが設定された場合のLLMの応答（`tool_calls` に指定ツールが含まれる）と、それに続くグラフの挙動の確認。
    *   この機能が利用できるLLMとそうでないLLMがあることの理解。

In [ ]:
# 解答欄004 - グラフ構築
from langchain_core.tools import tool # 解答例より (既にインポート済みだが明示)
from ____.____ import ____, ____ # 解答例より (既にインポート済みだが明示)
from ____.____.message import ____ # 解答例より (既にインポート済みだが明示)
from langchain_core.messages import ____, ____, ____, ToolMessage, ToolCall # 解答例より (既にインポート済みだが明示)
from typing import TypedDict, Annotated, List # 解答例より (既にインポート済みだが明示)
from uuid import uuid4 # 解答例より (既にインポート済みだが明示)
from langgraph.prebuilt import ToolNode # 解答例より (既にインポート済みだが明示)

# square_number ツールは問題001で定義済み

# 状態定義 (BasicToolAgentStateを再利用)
# class BasicToolAgentState(TypedDict): messages: Annotated[list, add_messages] (解答例よりコメントアウト)

# ノード定義
def forced_tool_agent_node(state: BasicToolAgentState):
    print("
[強制ツール呼び出しエージェントノード]")
    current_messages = state["messages"]
    response = None
    # tool_choice_arg = {"type": "tool", "name": "square_number"} # OpenAIの tool_choice 形式 (解答例より)

    if hasattr(llm, 'bind_tools') and LLM_PROVIDER == "openai": # 解答例の条件分岐を参考
        try:
            llm_force_square = llm.bind_tools([square_number], tool_choice="square_number") # 強制するツール名を指定 (解答例より)
            print(f"  強制ツール呼び出し[square_number]を設定してLLMを実行します。")
            response = llm_force_square.invoke(current_messages)
        except Exception as e:
            print(f"  WARN: tool_choice='square_number' でエラー: {e}。詳細な辞書形式で再試行します。") # 解答例より
            try:
                 llm_force_square = llm.bind_tools([square_number], tool_choice={"name": "square_number"}) # 解答例より
                 response = llm_force_square.invoke(current_messages)
            except Exception as e2:
                print(f"  ERROR: 詳細な辞書形式でもエラー: {e2}。フォールバックします。") # 解答例より
                print("  Fallback: square_number(99) を強制的に呼び出します。 (入力無視)") # 解答例より
                response = AIMessage(content="", tool_calls=[ToolCall(name="square_number", args={"number": 99.0}, id=f"ftc_forced_sq_{uuid4()[:4]}")]) # 解答例より
    elif hasattr(llm, 'bind_tools'): 
        print("  WARN: このLLMはtool_choiceによる強制ツール呼び出しを直接サポートしていない可能性があります。フォールバックします。") # 解答例より
        print("  Fallback: square_number(99) を強制的に呼び出します。 (入力無視)")
        response = AIMessage(content="", tool_calls=[ToolCall(name="square_number", args={"number": 99.0}, id=f"ftc_forced_sq_{uuid4()[:4]}")])
    else: 
        print("  ERROR: このLLMはツールをバインドできません。強制呼び出しは不可能です。フォールバックします。") # 解答例より
        print("  Fallback: square_number(99) を強制的に呼び出します。 (入力無視)") # 解答例より
        response = AIMessage(content="", tool_calls=[ToolCall(name="square_number", args={"number": 99.0}, id=f"ftc_forced_sq_{uuid4()[:4]}")]) # 解答例より

    print(f"  エージェント応答: {response}")
    return {"messages": [response]}

tool_node_q4 = ToolNode([square_number]) # square_numberのみを使用

# ルーター関数 (router_functionを再利用)
# def router_function(state: BasicToolAgentState) -> str: ... (解答例よりコメントアウト)

# グラフ構築
workflow_q4 = StateGraph(BasicToolAgentState)
workflow_q4.add_node("agent", forced_tool_agent_node)
workflow_q4.add_node("tools", tool_node_q4)

workflow_q4.set_entry_point("agent")
workflow_q4.add_conditional_edges("agent", router_function, {"call_tools": "tools", "__end__": END})
workflow_q4.add_edge("tools", "agent")

graph_q4 = workflow_q4.compile()

In [ ]:
# 解答欄004 - グラフ可視化
____ IPython.display ____ Image, display # 解答例より display をインポート

____:
    display(Image(graph_q4.get_graph().draw_png()))
____ Exception as e:
    print(f"グラフ描画に失敗: {e}")

In [ ]:
# 解答欄004 - グラフ実行
print("
--- 強制ツール呼び出しテスト (入力は無視され、square_numberが呼ばれるはず) ---")
initial_state_q4 = {"messages": [____(content="こんにちは！今日の天気はどうですか？")]} # 入力内容は強制呼び出しにより無視される想定
thread_q4 = {"configurable": {"thread_id": f"test-thread-q4-{uuid4()[:4]}"}}}
final_content_q4 = "(応答なし)"
____ event ____ graph_q4.____(initial_state_q4, config=thread_q4, recursion_limit=5): # 解答例より config 追加, recursion_limit は元の解答から
    print(f"Event: {event}")
    ____ 'agent' ____ event: # 解答例より
        agent_messages = event['agent'].get('messages', [])
        ____ agent_messages ____ isinstance(agent_messages[0], ____) and not agent_messages[0].____:
            final_content_q4 = agent_messages[0].content
    print("----");
print(f"最終応答内容: {final_content_q4}")

if "9801" in final_content_q4 or (LLM_PROVIDER == "openai" and final_content_q4): # OpenAIは引数をLLMが生成するので99とは限らない (解答例より)
    print(f"テスト成功の可能性: 応答にsquare_numberの結果らしきものが含まれています。 (LLMプロバイダ: {LLM_PROVIDER})") # 解答例より
else:
    print(f"テスト失敗/確認要: 応答に期待した結果が含まれていません。実際の応答: {final_content_q4} (LLMプロバイダ: {LLM_PROVIDER})") # 解答例より

<details><summary>解答004</summary>

``````python
from langchain_core.tools import tool
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, ToolMessage, ToolCall
from typing import TypedDict, Annotated, List
from uuid import uuid4
from IPython.display import Image, display
from langgraph.prebuilt import ToolNode

# square_number ツールは問題001で定義済み

# 状態定義 (BasicToolAgentStateを再利用)
# class BasicToolAgentState(TypedDict): messages: Annotated[list, add_messages]

# ノード定義
def forced_tool_agent_node(state: BasicToolAgentState):
    print("
[強制ツール呼び出しエージェントノード]")
    current_messages = state["messages"]
    response = None
    tool_choice_arg = {"type": "tool", "name": "square_number"} # OpenAIの tool_choice 形式

    # OpenAIモデルなど、tool_choiceをサポートするLLMの場合
    # (注意: llm.bind_tools の tool_choice 引数は LangChain のバージョンやモデルによって挙動が異なる場合がある)
    #   ChatOpenAI の場合は invoke の引数 tool_choice で渡すのがより確実な場合もある。
    #   llm.invoke(current_messages, tool_choice={"type": "function", "function": {"name": "square_number"}})
    #   ここでは bind_tools の tool_choice を試みる。
    if hasattr(llm, 'bind_tools') and LLM_PROVIDER == "openai": 
        try:
            # tool_choiceにツール名を直接渡すか、より詳細な辞書形式で渡す (モデル/バージョン依存)
            # LangChainの ChatOpenAI では tool_choice="square_number" のような文字列指定も可能だったり、
            # あるいは {"name": "square_number"} のような辞書を期待する場合もある。
            # 最新のOpenAI API仕様に準拠した形は {"type": "function", "function": {"name": "tool_name"}} だが、
            # LangChainの抽象化レイヤーで扱いやすいように調整されていることが多い。
            # ここでは最も一般的と思われるツール名直接指定を試みる。
            llm_force_square = llm.bind_tools([square_number], tool_choice="square_number")
            print(f"  強制ツール呼び出し[square_number]を設定してLLMを実行します。")
            response = llm_force_square.invoke(current_messages)
        except Exception as e:
            print(f"  WARN: tool_choice='square_number' でエラー: {e}。詳細な辞書形式で再試行します。")
            try:
                 llm_force_square = llm.bind_tools([square_number], tool_choice={"name": "square_number"})
                 response = llm_force_square.invoke(current_messages)
            except Exception as e2:
                print(f"  ERROR: 詳細な辞書形式でもエラー: {e2}。フォールバックします。")
                # フォールバック: OpenAI以外またはエラー時
                print("  Fallback: square_number(99) を強制的に呼び出します。 (入力無視)")
                response = AIMessage(content="", tool_calls=[ToolCall(name="square_number", args={"number": 99.0}, id=f"ftc_forced_sq_{uuid4()[:4]}")])
    
    elif hasattr(llm, 'bind_tools'): # OpenAI以外のLLMでtool_choiceがない場合
        print("  WARN: このLLMはtool_choiceによる強制ツール呼び出しを直接サポートしていない可能性があります。フォールバックします。")
        print("  Fallback: square_number(99) を強制的に呼び出します。 (入力無視)")
        response = AIMessage(content="", tool_calls=[ToolCall(name="square_number", args={"number": 99.0}, id=f"ftc_forced_sq_{uuid4()[:4]}")])
    else: # bind_tools もない場合
        print("  ERROR: このLLMはツールをバインドできません。強制呼び出しは不可能です。フォールバックします。")
        # response = AIMessage(content="エラー: ツール利用不可のため強制呼び出しできません。")
        print("  Fallback: square_number(99) を強制的に呼び出します。 (入力無視)") # エラーよりはダミー実行
        response = AIMessage(content="", tool_calls=[ToolCall(name="square_number", args={"number": 99.0}, id=f"ftc_forced_sq_{uuid4()[:4]}")])

    print(f"  エージェント応答: {response}")
    return {"messages": [response]}

tool_node_q4 = ToolNode([square_number])

# ルーター関数 (router_functionを再利用)
# def router_function(state: BasicToolAgentState) -> str: ... (問題001, 002と同じ)

# グラフ構築
workflow_q4 = StateGraph(BasicToolAgentState)
workflow_q4.add_node("agent", forced_tool_agent_node)
workflow_q4.add_node("tools", tool_node_q4)

workflow_q4.set_entry_point("agent")
workflow_q4.add_conditional_edges("agent", router_function, {"call_tools": "tools", "__end__": END})
workflow_q4.add_edge("tools", "agent")

graph_q4 = workflow_q4.compile()
try:
    display(Image(graph_q4.get_graph().draw_png()))
except Exception as e:
    print(f"グラフ描画に失敗: {e}")

# 実行テスト
print("
--- 強制ツール呼び出しテスト (入力は無視され、square_numberが呼ばれるはず) ---")
initial_state_q4 = {"messages": [HumanMessage(content="こんにちは！今日の天気はどうですか？")]}
thread_q4 = {"configurable": {"thread_id": f"test-thread-q4-{uuid4()[:4]}"}}}
final_content_q4 = "(応答なし)"
for event in graph_q4.stream(initial_state_q4, config=thread_q4, recursion_limit=5):
    print(f"Event: {event}")
    if 'agent' in event:
        agent_messages = event['agent'].get('messages', [])
        if agent_messages and isinstance(agent_messages[0], AIMessage) and not agent_messages[0].tool_calls:
            final_content_q4 = agent_messages[0].content
    print("----");
print(f"最終応答内容: {final_content_q4}")

if "9801" in final_content_q4 or (LLM_PROVIDER == "openai" and final_content_q4): # OpenAIは引数をLLMが生成するので99とは限らない
    print(f"テスト成功の可能性: 応答にsquare_numberの結果らしきものが含まれています。 (LLMプロバイダ: {LLM_PROVIDER})")
else:
    print(f"テスト失敗/確認要: 応答に期待した結果が含まれていません。実際の応答: {final_content_q4} (LLMプロバイダ: {LLM_PROVIDER})")
``````
</details>

<details><summary>解説004</summary>

#### この問題のポイント

*   **`tool_choice`パラメータ:** OpenAIのモデル（`gpt-3.5-turbo`以降など）では、LLM呼び出し時に`tool_choice`パラメータを指定することで、特定のツールの使用を強制したり、あるいはツール使用を禁止したり（`"none"`）、LLMに自動選択させたり（`"auto"`、デフォルト）することができます。
    *   特定のツールを強制する場合、`tool_choice="<ツール名>"` や `tool_choice={"type": "function", "function": {"name": "<ツール名>"}}` のような形式で指定します（正確な形式はLangChainのバージョンやOpenAI APIの仕様変更に注意してください）。
    *   この解答例では、`llm.bind_tools([square_number], tool_choice="square_number")` のように、`bind_tools`の引数として`tool_choice`を渡すことを試みています。これが機能すれば、LLMは入力メッセージの内容に関わらず、`square_number`ツールを呼び出すための`tool_calls`を生成しようとします（引数`number`の値はLLMが文脈から適当に判断するか、デフォルト値を使うなどします）。
*   **LLMによる引数の補完:** ツール呼び出しを強制された場合でも、そのツールが必要とする引数はLLMが現在の会話履歴や文脈から推測して補完しようとします。もし文脈から適切な引数が得られない場合は、LLMがデフォルト値や適当な値を設定することがあります（例: `square_number`なら適当な数値）。
*   **対応していないLLMの場合:** `tool_choice`のような強制呼び出し機能は、全てのLLMでサポートされているわけではありません。サポートされていないLLMの場合、この設定は無視されるか、エラーになることがあります。この解答例のフォールバック処理では、そのような場合に手動で特定のツールコール（`square_number(99.0)`）を生成して、グラフのフロー自体は追えるようにしています。
*   **ユースケース:**
    *   特定の分析ツールを定期的に実行させたい場合。
    *   ユーザーの意図が曖昧でも、まず特定の情報収集ツールを使わせたい場合。
    *   マルチモーダル入力で、画像が提供されたら必ず画像解析ツールを呼び出す、など。
*   **注意点:** 強制ツール呼び出しは便利な機能ですが、LLMの自律的な判断を妨げることにもなるため、その利用は慎重に検討する必要があります。タスクの性質やエージェントの設計思想に合わせて使い分けることが重要です。

---</details>

### ■ 問題005: 複数ツールの並列実行

LLMが一度の応答で複数の異なるツール呼び出しを要求し、それらのツールを並列で実行させたい場合があります（例: 「東京の天気と、大阪の天気の両方を調べて」）。LangGraphの`ToolNode`は、デフォルトで複数のツールコールを並列に実行できます。この問題では、LLMが一つのリクエストに対して複数のツール（例: 2つの異なる検索クエリ）を同時に呼び出すように促し、それらが並行して処理され、結果が統合されてLLMに返される流れを確認します。

*   **学習内容:**
    *   LLMが応答の`tool_calls`リストに複数のツール呼び出しを含めるようにプロンプト等で仕向ける方法（または、対応モデルの自然な振る舞いとして期待）。
    *   `ToolNode`が複数のツール呼び出しをどのように処理するか（通常は並列実行）。
    *   複数のツール実行結果（複数の`ToolMessage`）がどのようにエージェントに返され、後続の処理に利用されるか。

In [ ]:
# 解答欄005 - グラフ構築
from langchain_core.tools import tool # 解答例より (既にインポート済みだが明示)
from ____.____ import ____, ____ # 解答例より (既にインポート済みだが明示)
from ____.____.message import ____ # 解答例より (既にインポート済みだが明示)
from langchain_core.messages import ____, ____, ____, ToolMessage, ToolCall # 解答例より (既にインポート済みだが明示)
from typing import TypedDict, Annotated, List # 解答例より (既にインポート済みだが明示)
from uuid import uuid4 # 解答例より (既にインポート済みだが明示)
from langgraph.prebuilt import ToolNode # 解答例より (既にインポート済みだが明示)
import re # 解答例より

# search_tool は準備セルで初期化済み

tools_q5 = [search_tool] # search_tool のみを使用 (複数の呼び出しをLLMに期待) (解答例より)

# 状態定義 (BasicToolAgentStateを再利用)
# class BasicToolAgentState(TypedDict): messages: Annotated[list, add_messages] (解答例よりコメントアウト)

# ノード定義
def parallel_tool_agent_node(state: BasicToolAgentState):
    print("
[並列ツール呼び出しエージェントノード]")
    current_messages = state["messages"]
    response = None

    if hasattr(llm, 'bind_tools') and LLM_PROVIDER != "fake" and hasattr(llm.bind_tools(tools_q5), 'model') and not "Fake" in llm.bind_tools(tools_q5).model.__class__.__name__ : # 解答例の条件分岐
        llm_with_search_q5 = llm.bind_tools(tools_q5) 
        print("  LLMに複数ツール呼び出しを期待して実行します。") # 解答例より
        response = llm_with_search_q5.invoke(current_messages)
    else:
        print("  WARN: LLMがbind_toolsをサポートしていないかFakeLLM、または並列呼び出しを期待できないモデルです。手動で複数ツールコールを模倣します。") # 解答例より
        last_message = current_messages[-1]
        if isinstance(last_message, HumanMessage) and ("と" in last_message.content or "and" in last_message.content.lower()) and ("天気" in last_message.content or "観光" in last_message.content or "情報" in last_message.content): # 解答例の条件分岐
            queries = []
            parts = re.split(r'[と、]', last_message.content) # 解答例より
            for part in parts: # 解答例より
                part = part.strip()
                if "天気" in part: queries.append(part)
                elif "観光" in part or "情報" in part : queries.append(part)
            
            if not queries and len(parts) > 0: queries = [p.strip() for p in parts if p.strip()] # 解答例より
            if not queries: queries.append(last_message.content) # 解答例より

            if len(queries) >= 1 and search_tool is not None: # 解答例より
                tool_calls_list = []
                for i, q in enumerate(queries[:2]): # 最大2つまでの並列呼び出しを模倣 (解答例より)
                    print(f"    FakeLLM: 検索クエリ「{q}」で{search_tool.name}を呼び出します。") # 解答例より
                    tool_calls_list.append(ToolCall(name=search_tool.name, args={"query": q}, id=f"ftc_parallel_{i}_{uuid4()[:4]}"))
                if tool_calls_list: # 解答例より
                    response = AIMessage(content=f"FakeLLM: {len(tool_calls_list)}件の情報を検索します。", tool_calls=tool_calls_list) # 解答例より
                else: # 解答例より
                    response = AIMessage(content="FakeLLM: 検索クエリを特定できませんでした。") # 解答例より
            else: # 解答例より
                 response = AIMessage(content="FakeLLM: 検索ツールが利用できないか、クエリがありません。") # 解答例より
        elif isinstance(last_message, ToolMessage):
            all_tool_messages = [m.content for m in current_messages if isinstance(m, ToolMessage)] # 解答例より
            response_content = f"FakeLLM: {len(all_tool_messages)}件のツール結果「{', '.join([res[:30] + '...' for res in all_tool_messages])}」を元に最終応答を生成します。" # 解答例より
            response = AIMessage(content=response_content) # 解答例より
        else:
            response = AIMessage(content="FakeLLM: 複数ツールの呼び出し条件にマッチしませんでした。通常の応答を返します。") # 解答例より

    print(f"  エージェント応答: {response}")
    return {"messages": [response]}

tool_node_q5 = ToolNode(tools_q5) # tools_q5 を使用 (解答例より)

# ルーター関数 (router_functionを再利用)
# def router_function(state: BasicToolAgentState) -> str: ... (解答例よりコメントアウト)

# グラフ構築
workflow_q5 = StateGraph(BasicToolAgentState)
workflow_q5.add_node("agent", parallel_tool_agent_node)
workflow_q5.add_node("tools", tool_node_q5)

workflow_q5.set_entry_point("agent")
workflow_q5.add_conditional_edges("agent", router_function, {"call_tools": "tools", "__end__": END})
workflow_q5.add_edge("tools", "agent")

graph_q5 = workflow_q5.compile()

In [ ]:
# 解答欄005 - グラフ可視化
____ IPython.display ____ Image, display # 解答例より display をインポート

____:
    display(Image(graph_q5.get_graph().draw_png()))
____ Exception as e:
    print(f"グラフ描画に失敗: {e}")

In [ ]:
# 解答欄005 - グラフ実行
query_q5 = "東京の天気と、京都の観光情報を同時に調べてください。"
____ (search_tool ____ ____ ____ (hasattr(search_tool, 'name') and search_tool.name == "dummy_search_tool")) and TAVILY_API_KEY is None:
    print("TAVILY_API_KEY が未設定のため、このテストはダミー応答になります。")

print(f"
--- 並列ツール呼び出しテスト (入力: {query_q5}) ---")
initial_state_q5 = {"messages": [____(content=query_q5)]}
thread_q5 = {"configurable": {"thread_id": f"test-thread-q5-{uuid4()[:4]}"}}}
final_content_q5 = "(応答なし)"
num_tool_calls_in_response = 0 # 解答例より

for event in graph_q5.____(initial_state_q5, config=thread_q5, recursion_limit=5): # 解答例より config 追加, recursion_limit は元の解答から
    print(f"Event: {event}")
    if 'agent' in event: # 解答例より
        agent_messages = event['agent'].get('messages', [])
        if agent_messages and isinstance(agent_messages[0], ____):
            if agent_messages[0].____:
                num_tool_calls_in_response = len(agent_messages[0].____)
                print(f"  エージェントが {num_tool_calls_in_response} 件のツール呼び出しを要求しました。")
            elif not agent_messages[0].____ : # 最終応答
                final_content_q5 = agent_messages[0].content
    if 'tools' in event: # 解答例より
        tool_messages = event['tools'].get('messages', [])
        if len(tool_messages) > 1:
            print(f"  ToolNodeが複数のToolMessage ({len(tool_messages)}件) を返しました。")
        elif tool_messages:
            print(f"  ToolNodeがToolMessageを返しました: {tool_messages[0].name if tool_messages else ''}")
    print("----");

print(f"
LLMが要求したツール呼び出しの数: {num_tool_calls_in_response}") # 解答例より
print(f"最終応答内容: {final_content_q5}")
if num_tool_calls_in_response >= 2 and LLM_PROVIDER != "fake": # 解答例より
    print("テスト成功の可能性(実際のLLMの場合): LLMが複数のツール呼び出しを要求しました。") # 解答例より
elif num_tool_calls_in_response >=1 and LLM_PROVIDER == "fake": # 解答例より
    print("テスト成功の可能性(FakeLLMの場合): FakeLLMが模倣によりツール呼び出しを生成しました。") # 解答例より
else: # 解答例より
    print(f"テスト確認要: LLMが期待通りに複数のツール呼び出しを要求しませんでした（実際の呼び出し数: {num_tool_calls_in_response}）。プロンプトやLLMの能力を確認してください。") # 解答例より

<details><summary>解答005</summary>

``````python
from langchain_core.tools import tool
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, ToolMessage, ToolCall
from typing import TypedDict, Annotated, List
from uuid import uuid4
from IPython.display import Image, display
from langgraph.prebuilt import ToolNode

# search_tool は準備セルで TavilySearchResults またはダミーとして初期化済み

tools_q5 = [search_tool] # search_tool のみを使用

# 状態定義 (BasicToolAgentStateを再利用)
# class BasicToolAgentState(TypedDict): messages: Annotated[list, add_messages]

# ノード定義
def parallel_tool_agent_node(state: BasicToolAgentState):
    print("
[並列ツール呼び出しエージェントノード]")
    current_messages = state["messages"]
    response = None

    if hasattr(llm, 'bind_tools') and LLM_PROVIDER != "fake" and hasattr(llm.bind_tools(tools_q5), 'model') and not "Fake" in llm.bind_tools(tools_q5).model.__class__.__name__ :
        # OpenAIのFunction Calling対応モデルなどは、プロンプト次第で複数のツールコールを返すことがある
        # 例: 「東京の天気と大阪の天気を調べて」
        # この挙動はLLMの能力に大きく依存する
        llm_with_search_q5 = llm.bind_tools(tools_q5) 
        print("  LLMに複数ツール呼び出しを期待して実行します。")
        response = llm_with_search_q5.invoke(current_messages)
    else:
        print("  WARN: LLMがbind_toolsをサポートしていないかFakeLLM、または並列呼び出しを期待できないモデルです。手動で複数ツールコールを模倣します。")
        last_message = current_messages[-1]
        if isinstance(last_message, HumanMessage) and ("と" in last_message.content or "and" in last_message.content.lower()) and ("天気" in last_message.content or "観光" in last_message.content or "情報" in last_message.content):
            queries = []
            # この模倣は非常に単純であり、実際のLLMのNLU能力とは異なります
            # 簡易的に「と」や「、」で分割してクエリ候補を生成
            parts = re.split(r'[と、]', last_message.content)
            for part in parts:
                part = part.strip()
                if "天気" in part: queries.append(part)
                elif "観光" in part or "情報" in part : queries.append(part)
            
            if not queries and len(parts) > 0: # 上手く抽出できなかった場合、partをそのまま使う
                queries = [p.strip() for p in parts if p.strip()]
            if not queries: queries.append(last_message.content) # それでもダメなら全体

            if len(queries) >= 1 and search_tool is not None:
                tool_calls_list = []
                for i, q in enumerate(queries[:2]): # 最大2つまでの並列呼び出しを模倣
                    print(f"    FakeLLM: 検索クエリ「{q}」で{search_tool.name}を呼び出します。")
                    tool_calls_list.append(ToolCall(name=search_tool.name, args={"query": q}, id=f"ftc_parallel_{i}_{uuid4()[:4]}"))
                if tool_calls_list:
                    response = AIMessage(content=f"FakeLLM: {len(tool_calls_list)}件の情報を検索します。", tool_calls=tool_calls_list)
                else:
                    response = AIMessage(content="FakeLLM: 検索クエリを特定できませんでした。")
            else:
                 response = AIMessage(content="FakeLLM: 検索ツールが利用できないか、クエリがありません。")
        elif isinstance(last_message, ToolMessage):
            # 実際には複数のToolMessageがmessagesリストに入ってくるので、それらを考慮して応答を生成する必要がある
            # ここでは最後のToolMessageだけを簡単に参照
            all_tool_messages = [m.content for m in current_messages if isinstance(m, ToolMessage)]
            response_content = f"FakeLLM: {len(all_tool_messages)}件のツール結果「{', '.join([res[:30] + '...' for res in all_tool_messages])}」を元に最終応答を生成します。"
            response = AIMessage(content=response_content)
        else:
            response = AIMessage(content="FakeLLM: 複数ツールの呼び出し条件にマッチしませんでした。通常の応答を返します。")

    print(f"  エージェント応答: {response}")
    return {"messages": [response]}

tool_node_q5 = ToolNode(tools_q5)

# ルーター関数 (router_functionを再利用)
# def router_function(state: BasicToolAgentState) -> str: ...

# グラフ構築
workflow_q5 = StateGraph(BasicToolAgentState)
workflow_q5.add_node("agent", parallel_tool_agent_node)
workflow_q5.add_node("tools", tool_node_q5)

workflow_q5.set_entry_point("agent")
workflow_q5.add_conditional_edges("agent", router_function, {"call_tools": "tools", "__end__": END})
workflow_q5.add_edge("tools", "agent")

graph_q5 = workflow_q5.compile()
try:
    display(Image(graph_q5.get_graph().draw_png()))
except Exception as e:
    print(f"グラフ描画に失敗: {e}")

# 実行テスト
query_q5 = "東京の天気と、京都の観光情報を同時に調べてください。"
if (search_tool is None or (hasattr(search_tool, 'name') and search_tool.name == "dummy_search_tool")) and TAVILY_API_KEY is None:
    print("TAVILY_API_KEY が未設定のため、このテストはダミー応答になります。")

print(f"
--- 並列ツール呼び出しテスト (入力: {query_q5}) ---")
initial_state_q5 = {"messages": [HumanMessage(content=query_q5)]}
thread_q5 = {"configurable": {"thread_id": f"test-thread-q5-{uuid4()[:4]}"}}}
final_content_q5 = "(応答なし)"
num_tool_calls_in_response = 0

for event in graph_q5.stream(initial_state_q5, config=thread_q5, recursion_limit=5):
    print(f"Event: {event}")
    if 'agent' in event:
        agent_messages = event['agent'].get('messages', [])
        if agent_messages and isinstance(agent_messages[0], AIMessage):
            if agent_messages[0].tool_calls:
                num_tool_calls_in_response = len(agent_messages[0].tool_calls)
                print(f"  エージェントが {num_tool_calls_in_response} 件のツール呼び出しを要求しました。")
            elif not agent_messages[0].tool_calls : # 最終応答
                final_content_q5 = agent_messages[0].content
    if 'tools' in event:
        tool_messages = event['tools'].get('messages', [])
        if len(tool_messages) > 1:
            print(f"  ToolNodeが複数のToolMessage ({len(tool_messages)}件) を返しました。")
        elif tool_messages:
            print(f"  ToolNodeがToolMessageを返しました: {tool_messages[0].name if tool_messages else ''}")
    print("----");

print(f"
LLMが要求したツール呼び出しの数: {num_tool_calls_in_response}")
print(f"最終応答内容: {final_content_q5}")
if num_tool_calls_in_response >= 2 and LLM_PROVIDER != "fake":
    print("テスト成功の可能性(実際のLLMの場合): LLMが複数のツール呼び出しを要求しました。")
elif num_tool_calls_in_response >=1 and LLM_PROVIDER == "fake":
    print("テスト成功の可能性(FakeLLMの場合): FakeLLMが模倣によりツール呼び出しを生成しました。")
else:
    print(f"テスト確認要: LLMが期待通りに複数のツール呼び出しを要求しませんでした（実際の呼び出し数: {num_tool_calls_in_response}）。プロンプトやLLMの能力を確認してください。")
``````
</details>

<details><summary>解説005</summary>

#### この問題のポイント

*   **LLMによる複数ツール呼び出しの要求:**
    *   OpenAIのFunction Calling対応モデルなど、一部の高度なLLMは、ユーザーのプロンプトが複数の情報要求を含んでいる場合（例: 「東京の天気と大阪の天気を教えて」）、応答の`tool_calls`リストに複数の`ToolCall`オブジェクトを含める能力があります。これにより、一度に複数のツール（または同じツールを異なる引数で複数回）を呼び出すよう指示できます。
    *   この挙動はLLMの能力とプロンプトの設計に大きく依存します。必ずしも全てのLLMが期待通りに複数のツールコールを生成するわけではありません。
    *   FakeLLMを使用している場合、この解答例のフォールバックでは、入力文字列に「と」や「天気」「観光」などのキーワードが含まれている場合に、手動で複数の`ToolCall`オブジェクトを生成して模倣しています。
*   **`ToolNode`による並列実行:** `ToolNode`は、受け取った`AIMessage`の`tool_calls`リストに含まれる複数のツール呼び出しを（通常は非同期に、つまり実質的に）並列で実行します。各ツール呼び出しは独立して処理され、それぞれの結果が個別の`ToolMessage`として生成されます。
*   **結果の集約と再処理:** `ToolNode`が生成した複数の`ToolMessage`は、リストとして次のノード（通常はエージェントノード）の状態の`messages`キーに追加されます。エージェントノードのLLMは、これらの複数のツール実行結果をすべて考慮して、最終的な統合された応答をユーザーに生成します。
*   **ユースケース:**
    *   複数の異なる情報源からのデータを一度に収集・比較する（例: 複数の都市の天気を同時に調べる、複数の株価を同時に取得する）。
    *   一つのタスクを複数のサブタスクに分割し、それぞれを異なるツールで並列処理する。
*   **注意点:** LLMが期待通りに複数のツールコールを生成するかどうかは、LLMのモデル、バージョン、プロンプトの書き方などに影響されます。また、あまりにも多くのツールコールを一度に要求すると、LLMのコンテキストウィンドウの制限や処理能力の限界を超える可能性もあります。

---</details>

### ■ 問題006: シンプルな Plan-and-Execute エージェントの構築

Plan-and-Executeは、エージェントが複雑なタスクに取り組むための一つの戦略です。まず「計画（Plan）」を立て、その計画に基づいて個々のステップを「実行（Execute）」していきます。この問題では、この戦略の非常にシンプルな形をLangGraphで実装します。
1.  **プランナーノード:** ユーザーの要求を受けて、それを達成するためのステップのリスト（計画）を生成します（LLMを使用）。
2.  **エグゼキューターノード:** 計画の各ステップを順番に（またはLLMの指示で）実行します。ここでは、ステップ実行のシミュレーションとして、簡単なツール（例: `square_number` や `reverse_string`、または単にステップ内容をログ出力するだけ）を使用します。
3.  計画の全ステップが完了したら終了します。

*   **学習内容:**
    *   タスクを複数のステップに分解する「プランナー」の役割と、各ステップを実行する「エグゼキューター」の役割を異なるノードに分離する考え方。
    *   状態（State）に計画全体、現在のステップ、実行結果などを保持し、それに基づいて処理を進める方法。
    *   簡単なループ構造（計画のステップを順に処理する）と条件分岐（計画が完了したかどうかの判断）の組み合わせ。

In [ ]:
# 解答欄006 - グラフ構築
from typing import ____, List, Optional, ____
from ____.____ import ____, ____
from ____.____.message import ____ # メッセージ履歴も活用する場合
from langchain_core.messages import ____, AIMessage, ToolMessage, ToolCall # ToolMessage, ToolCallは直接は使わないが概念として (解答例より)
from langchain_core.tools import tool
import json
from uuid import uuid4 # 解答例より

# --- ツールの準備 (既存のものを再利用または新規定義) ---
# @tool def simple_math_tool(expression: str) -> str: ... (例)
# ここでは、square_number と reverse_string を使う想定
available_tools_for_plan = {t.name: t for t in [square_number, reverse_string]}

# --- 状態定義 ---
class PlanExecuteState(TypedDict):
    user_request: str
    plan: Optional[List[dict]] # 例: [{'tool_name': 'square_number', 'args': {'number': 5}}, ...]
    current_step_index: int
    step_results: List[str] # 各ステップの実行結果(ToolMessageの内容など)
    final_answer: Optional[str]
    messages: Annotated[list, add_messages] # 会話ログ用

# --- ノード定義 ---
def planner_node(state: PlanExecuteState):
    print(f"
[プランナーノード] ユーザーリクエスト: {state['user_request']}")
    request = state['user_request'].lower()
    generated_plan = []
    import re # 解答例より
    
    if "二乗" in request and "逆順" in request:
        num_match = re.search(r'(\d+\.?\d*|\.\d+)', request) # 解答例より
        num_for_square = float(num_match.group(1)) if num_match else 0.0 # 解答例より
        generated_plan.append({"tool_name": "square_number", "args": {"number": num_for_square}, "step_description": f"{num_for_square}を二乗する"})
        generated_plan.append({"tool_name": "reverse_string", "args": {"text": "<前のステップの結果>"}, "step_description": "前のステップの結果を逆順にする"})
    elif "二乗" in request:
        num_match = re.search(r'(\d+\.?\d*|\.\d+)', request) # 解答例より
        num_for_square = float(num_match.group(1)) if num_match else 0.0 # 解答例より
        generated_plan.append({"tool_name": "square_number", "args": {"number": num_for_square}, "step_description": f"{num_for_square}を二乗する"})
    elif "逆順" in request: # 解答例より
        str_match = re.search(r"['"]([^'"]*)['"]", request) # 解答例より
        text_to_rev = str_match.group(1) if str_match else request.replace("逆順に","").replace("逆順","").strip() # 解答例より
        generated_plan.append({"tool_name": "reverse_string", "args": {"text": text_to_rev}, "step_description": f"「{text_to_rev}」を逆順にする"}) # 解答例より
    else:
        generated_plan.append({"tool_name": "direct_answer", "args": {"text": "計画を立てられませんでした。"}, "step_description": "直接応答"})
        
    print(f"  生成された計画: {generated_plan}")
    return {"plan": generated_plan, "current_step_index": 0, "step_results": [], "messages": [AIMessage(content=f"計画を立てました: {json.dumps(generated_plan)}")]}

def executor_node(state: PlanExecuteState): # 解答例では updated_executor_node は不要で executor_node 内で idx 更新
    plan = state["plan"]
    idx = state["current_step_index"]
    
    if not plan or idx >= len(plan):
        print("[エグゼキューターノード] 実行すべきステップがありません。") # 解答例より
        return {"current_step_index": idx} # 変更なし (解答例より)
    
    current_step = plan[idx]
    tool_name = current_step["tool_name"]
    tool_args = current_step["args"].copy() 
    step_desc = current_step["step_description"]
    print(f"
[エグゼキューターノード] ステップ {idx + 1}/{len(plan)}: {step_desc} (ツール: {tool_name}, 引数: {tool_args})")

    if tool_args.get("text") == "<前のステップの結果>" and state["step_results"]:
        tool_args["text"] = str(state["step_results"][-1])
        print(f"    引数更新: text='{tool_args['text']}'")

    result_content = ""
    if tool_name in available_tools_for_plan:
        selected_tool = available_tools_for_plan[tool_name]
        try:
            tool_output = selected_tool.invoke(tool_args)
            result_content = str(tool_output)
            print(f"  ツール実行結果: {result_content}")
        except Exception as e:
            result_content = f"ツール実行エラー: {e}"
            print(f"  エラー: {result_content}")
    elif tool_name == "direct_answer":
        result_content = tool_args.get("text", "エラー: direct_answerにtextがありません") # 解答例より
        print(f"  直接応答: {result_content}")
    else:
        result_content = f"不明なツール: {tool_name}"
        print(f"  エラー: {result_content}")
        
    updated_step_results = state["step_results"] + [result_content]
    simulated_tool_message = ToolMessage(content=result_content, tool_call_id=f"plan_step_{idx}", name=tool_name)
    return {
        "step_results": updated_step_results, 
        "messages": [simulated_tool_message],
        "current_step_index": idx + 1  # 次のステップに進むためにインデックスを更新 (解答例より)
    }

def final_answer_node(state: PlanExecuteState):
    print("
[最終回答生成ノード]")
    if state["step_results"]:
        answer = f"計画の最終結果: {state['step_results'][-1]}"
    else:
        answer = "計画が実行されなかったか、結果がありませんでした。"
    print(f"  最終回答: {answer}")
    return {"final_answer": answer, "messages": [AIMessage(content=answer)]}

# --- ルーター関数 ---
def route_plan_execution(state: PlanExecuteState):
    plan = state.get("plan", [])
    current_idx = state.get("current_step_index", 0)
    
    if current_idx < len(plan):
        print(f"  -> ルーター: 次のステップ {current_idx + 1} を実行します。")
        return "execute_next_step"
    else:
        print("  -> ルーター: 全ステップ完了。最終回答へ。")
        return "generate_final_answer" # 最終回答生成へのキー (解答例より)

# --- グラフ構築 ---
workflow_q6 = StateGraph(PlanExecuteState)
workflow_q6.add_node("planner", planner_node)
workflow_q6.add_node("executor", executor_node) # updated_executor_nodeは不要、executor_node内でidx更新 (解答例より)
workflow_q6.add_node("final_answerer", final_answer_node)

workflow_q6.set_entry_point("planner")
workflow_q6.add_edge("planner", "executor") 

workflow_q6.add_conditional_edges(
    "executor",
    route_plan_execution,
    {
        "execute_next_step": "executor", 
        "generate_final_answer": "final_answerer" # 解答例より
    }
)
workflow_q6.add_edge("final_answerer", END)

graph_q6 = workflow_q6.compile()

In [ ]:
# 解答欄006 - グラフ可視化
____ IPython.display ____ Image, display # 解答例より display をインポート

____:
    display(Image(graph_q6.get_graph().draw_png()))
____ Exception as e:
    print(f"グラフ描画に失敗: {e}")

In [ ]:
# 解答欄006 - グラフ実行
request_q6_complex = "数値10を二乗して、その結果の数値を文字列にして逆順にしてください。"
request_q6_simple_sq = "数値7を二乗してください。"
request_q6_simple_rev = "'apple'を逆順にしてください。" # 解答例より
request_q6_unknown = "今日の天気は？" # 解答例より

____ req_idx, user_req ____ enumerate([request_q6_complex, request_q6_simple_sq, request_q6_simple_rev, request_q6_unknown]): # 解答例よりテストケース追加
    print(f"
--- Plan-____-Executeテスト {req_idx + 1} (リクエスト: {user_req}) ---")
    initial_state_q6 = {
        "user_request": user_req,
        "messages": [____(content=user_req)],
        "plan": ____, "current_step_index":0, "step_results": [], "final_answer": ____
    }
    thread_q6 = {"configurable": {"thread_id": f"test-thread-q6-{req_idx}-{uuid4()[:4]}"}}}
    final_state_q6_val = ____ # 解答例より
    ____ event in graph_q6.____(initial_state_q6, config=thread_q6, recursion_limit=10): # 解答例より recursion_limit 変更
        print(f"Event: {event}")
        print("----");
    
    final_state_q6_full = graph_q6.____(config=thread_q6) # 解答例より
    if final_state_q6_full: # 解答例より
        final_state_q6_val = final_state_q6_full.values
        print(f"最終回答: {final_state_q6_val.get('final_answer')}")
        print(f"全ステップ結果: {final_state_q6_val.get('step_results')}")
    else:
        print("最終状態が取得できませんでした。")

<details><summary>解答006</summary>

``````python
from typing import TypedDict, List, Optional, Annotated
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages 
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage, ToolCall 
from langchain_core.tools import tool
import json
from uuid import uuid4
from IPython.display import Image, display

# --- ツールの準備 (問題001, 002から再利用) ---
# @tool def square_number(number: float) -> float: ...
# @tool def reverse_string(text: str) -> str: ...
available_tools_for_plan = {t.name: t for t in [square_number, reverse_string]}

# --- 状態定義 ---
class PlanExecuteState(TypedDict):
    user_request: str
    plan: Optional[List[dict]] 
    current_step_index: int
    step_results: List[str] 
    final_answer: Optional[str]
    messages: Annotated[list, add_messages]

# --- ノード定義 ---
def planner_node(state: PlanExecuteState):
    print(f"
[プランナーノード] ユーザーリクエスト: {state['user_request']}")
    request = state['user_request'].lower()
    generated_plan = []
    import re
    
    # このダミープランナーは非常に単純化されています。
    # 実際のLLMはもっと複雑なリクエストを解釈し、適切なツールと引数で計画を立てます。
    if "二乗" in request and "逆順" in request:
        num_match = re.search(r'(\d+\.?\d*|\.\d+)', request)
        num_for_square = float(num_match.group(1)) if num_match else 0.0
        generated_plan.append({"tool_name": "square_number", "args": {"number": num_for_square}, "step_description": f"{num_for_square}を二乗する"})
        generated_plan.append({"tool_name": "reverse_string", "args": {"text": "<前のステップの結果>"}, "step_description": "前のステップの結果を逆順にする"})
    elif "二乗" in request:
        num_match = re.search(r'(\d+\.?\d*|\.\d+)', request)
        num_for_square = float(num_match.group(1)) if num_match else 0.0
        generated_plan.append({"tool_name": "square_number", "args": {"number": num_for_square}, "step_description": f"{num_for_square}を二乗する"})
    elif "逆順" in request:
        str_match = re.search(r"['"]([^'"]*)['"]", request) # クォートされた文字列
        text_to_rev = str_match.group(1) if str_match else request.replace("逆順に","").replace("逆順","").strip()
        generated_plan.append({"tool_name": "reverse_string", "args": {"text": text_to_rev}, "step_description": f"「{text_to_rev}」を逆順にする"})
    else:
        generated_plan.append({"tool_name": "direct_answer", "args": {"text": "計画を立てられませんでした。"}, "step_description": "直接応答"})
        
    print(f"  生成された計画: {generated_plan}")
    return {"plan": generated_plan, "current_step_index": 0, "step_results": [], "messages": [AIMessage(content=f"計画を立てました: {json.dumps(generated_plan)}")]}

def executor_node(state: PlanExecuteState):
    plan = state["plan"]
    idx = state["current_step_index"]
    
    if not plan or idx >= len(plan):
        print("[エグゼキューターノード] 実行すべきステップがありません。")
        return {"current_step_index": idx} 
    
    current_step = plan[idx]
    tool_name = current_step["tool_name"]
    tool_args = current_step["args"].copy() 
    step_desc = current_step["step_description"]
    print(f"
[エグゼキューターノード] ステップ {idx + 1}/{len(plan)}: {step_desc} (ツール: {tool_name}, 引数: {tool_args})")

    if tool_args.get("text") == "<前のステップの結果>" and state["step_results"]:
        tool_args["text"] = str(state["step_results"][-1])
        print(f"    引数更新: text='{tool_args['text']}'")

    result_content = ""
    if tool_name in available_tools_for_plan:
        selected_tool = available_tools_for_plan[tool_name]
        try:
            tool_output = selected_tool.invoke(tool_args)
            result_content = str(tool_output)
            print(f"  ツール実行結果: {result_content}")
        except Exception as e:
            result_content = f"ツール実行エラー: {e}"
            print(f"  エラー: {result_content}")
    elif tool_name == "direct_answer":
        result_content = tool_args.get("text", "エラー: direct_answerにtextがありません")
        print(f"  直接応答: {result_content}")
    else:
        result_content = f"不明なツール: {tool_name}"
        print(f"  エラー: {result_content}")
        
    updated_step_results = state["step_results"] + [result_content]
    simulated_tool_message = ToolMessage(content=result_content, tool_call_id=f"plan_step_{idx}", name=tool_name)
    return {
        "step_results": updated_step_results, 
        "messages": [simulated_tool_message],
        "current_step_index": idx + 1  # 次のステップに進むためにインデックスを更新
    }

def final_answer_node(state: PlanExecuteState):
    print("
[最終回答生成ノード]")
    if state["step_results"]:
        answer = f"計画の最終結果: {state['step_results'][-1]}"
    else:
        answer = "計画が実行されなかったか、結果がありませんでした。"
    print(f"  最終回答: {answer}")
    return {"final_answer": answer, "messages": [AIMessage(content=answer)]}

# --- ルーター関数 ---
def route_plan_execution(state: PlanExecuteState):
    plan = state.get("plan", [])
    current_idx = state.get("current_step_index", 0)
    
    if current_idx < len(plan):
        print(f"  -> ルーター: 次のステップ {current_idx + 1} を実行します。")
        return "execute_next_step"
    else:
        print("  -> ルーター: 全ステップ完了。最終回答へ。")
        return "generate_final_answer" 

# --- グラフ構築 ---
workflow_q6 = StateGraph(PlanExecuteState)
workflow_q6.add_node("planner", planner_node)
workflow_q6.add_node("executor", executor_node) # updated_executor_nodeは不要、executor_node内でidx更新
workflow_q6.add_node("final_answerer", final_answer_node)

workflow_q6.set_entry_point("planner")
workflow_q6.add_edge("planner", "executor") 

workflow_q6.add_conditional_edges(
    "executor",
    route_plan_execution,
    {
        "execute_next_step": "executor", 
        "generate_final_answer": "final_answerer" 
    }
)
workflow_q6.add_edge("final_answerer", END)

graph_q6 = workflow_q6.compile()
try:
    display(Image(graph_q6.get_graph().draw_png()))
except Exception as e:
    print(f"グラフ描画に失敗: {e}")

# 実行テスト
request_q6_complex = "数値10を二乗して、その結果の数値を文字列にして逆順にしてください。"
request_q6_simple_sq = "数値7を二乗してください。"
request_q6_simple_rev = "'apple'を逆順にしてください。"
request_q6_unknown = "今日の天気は？"

for req_idx, user_req in enumerate([request_q6_complex, request_q6_simple_sq, request_q6_simple_rev, request_q6_unknown]):
    print(f"
--- Plan-and-Executeテスト {req_idx + 1} (リクエスト: {user_req}) ---")
    initial_state_q6 = {
        "user_request": user_req,
        "messages": [HumanMessage(content=user_req)],
        "plan": None, "current_step_index":0, "step_results": [], "final_answer": None
    }
    thread_q6 = {"configurable": {"thread_id": f"test-thread-q6-{req_idx}-{uuid4()[:4]}"}}}
    final_state_q6_val = None
    for event in graph_q6.stream(initial_state_q6, config=thread_q6, recursion_limit=10):
        print(f"Event: {event}")
        # streamの最後のイベントが最終状態を含むとは限らない (ENDの場合など)
        # get_stateで取得するのが確実
        print("----");
    
    final_state_q6_full = graph_q6.get_state(config=thread_q6)
    if final_state_q6_full:
        final_state_q6_val = final_state_q6_full.values
        print(f"最終回答: {final_state_q6_val.get('final_answer')}")
        print(f"全ステップ結果: {final_state_q6_val.get('step_results')}")
    else:
        print("最終状態が取得できませんでした。")
``````
</details>

<details><summary>解説006</summary>

#### この問題のポイント

*   **Plan-and-Executeの概念:**
    *   **プランナー (`planner_node`):** ユーザーの要求を受け取り、それを達成するための一連のステップ（計画）を立案します。この解答例では、LLMの代わりに単純なルールベースで計画（ツールのリストと引数）を生成していますが、実際の高度なエージェントではここがLLMの役割となり、より複雑な推論を行います。
    *   **エグゼキューター (`executor_node`):** プランナーが立てた計画の各ステップを順番に実行します。各ステップは通常、特定のツール呼び出しに対応します。このノードは、現在のステップで指定されたツールを実行し、その結果を状態に記録します。
*   **状態管理 (`PlanExecuteState`):**
    *   `user_request`: 最初のユーザーの要求。
    *   `plan`: プランナーが生成したステップのリスト。各ステップは辞書で、使用するツール名、引数、説明などを含みます。
    *   `current_step_index`: 現在実行中の計画ステップのインデックス。
    *   `step_results`: 各ステップの実行結果を格納するリスト。
    *   `final_answer`: 全ステップ完了後に生成される最終的な回答。
    *   `messages`: デバッグやログ、あるいはLLMへの入力として会話履歴を保持します。
*   **グラフのフロー:**
    1.  `planner_node` が計画を生成し、`current_step_index` を0に初期化します。
    2.  `executor_node` が `plan[current_step_index]` を実行します。
        *   ツール実行結果を `step_results` に追加します。
        *   `current_step_index` をインクリメントします。
    3.  `route_plan_execution` ルーターが `current_step_index` と `plan` の長さを比較します。
        *   まだ実行すべきステップが残っていれば、再び `executor_node` に処理を戻します（ループ）。
        *   全ステップが完了していれば、`final_answer_node` に処理を移します。
    4.  `final_answer_node` が `step_results` をもとに最終回答を生成し、`END` で終了します。
*   **簡易的な実装:** この解答例のプランナーとエグゼキューターは、実際のPlan-and-Executeエージェントと比較して非常に単純化されています。
    *   プランナーは固定的なロジックで、LLMの動的な計画能力はありません。
    *   エグゼキューターは、前のステップの結果を次のステップの引数として単純に渡す処理（`"<前のステップの結果>"`）しか行っていません。実際のシステムでは、より複雑な依存関係の解決やエラーハンドリングが必要です。
    *   ツール自体も簡単なものです。
    しかし、この基本的な骨組みは、より高度なPlan-and-Executeエージェントを構築する上での出発点となります。

---</details>